In [2]:
import numpy as np  # probably don't need to load
import pandas as pd
#import datetime as dt
#import pandas_datareader.data as web  # probably don't need to load
#import quandl

#import blpapi
#from xbbg import blp
#import CLOutilsPyXLL as clo
import CLOComplianceUtils as ccu

#import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# The Code

In [17]:
def Port_stats(model_df, weight_col='Par_no_default',format_output=True):
    """
    Arg in:
        model_df
        #ind_avg_eu: table (df) Moody's discrete lookup table that maps AIEUS to IDS
    
    - Estimated Libor
    - Minimum Floating Spread Test - Without Libor Floors
    - Minimum Floating Spread Test - WithLibor Floors (adj. All in Rate)
    - Maximum Moody's Rating Factor Test
    - Maximum Moody's Rating Factor Test (NEW WARF)
    - Maximum Moody's Rating Factor Test (Orig WARF)
    - Minimum Weighted Average Moody's Recovery Rate Test
    - Minimum Weighted Average S&P Recovery Rate Class A-1a
    - Moody's Diversity Test
    - WAP (Current Positions use Actual purchase price, all others use Ask price)
    - Total Portfolio Par (excluding Defaults)
    - Total Portfolio Par
    - Current Portfolio 
    
    
    """
    
    Port_stats_df = pd.DataFrame(np.nan,index=['Min Floating Spread Test - no Libor Floors',
        'Min Floating Spread Test - With Libor Floors',
        'Max Moodys Rating Factor Test (NEW WARF)',
        'Max Moodys Rating Factor Test (Orig WARF)',
        'Min Moodys Recovery Rate Test',
        'Min S&P Recovery Rate Class A-1a',
        'Moodys Diversity Test',
    #    'WAP',
        'Weighted Average Life Test',
        'Percent CCC & lower',
        'Percent 2nd Lien',
    #    'Percent Sub80',
    #    'Percent Sub90',
    #    'Percent CovLite',
    #    'Pot Par B/L Sale',
    #    'Pot Par B/L Buy',
    #    'Pot Par B/L Total',
        'Total Portfolio Par (excl. Defaults)',
    #    'Total Portfolio Par',
     #   'Current Portfolio'
                                              ],columns = [weight_col])

    
    # Par_no_default works for current positions only
    # if trying to compare potential new trades, need updated field
    # also Total isn't dynamically updated for Potential Trades once
    # model_df is created.  Need to find solution
    
    Port_stats_df.loc['Min Floating Spread Test - no Libor Floors',weight_col] = \
        Weighted_Average_Spread(model_df,weight_col,libor=.002)*100
    
    Port_stats_df.loc['Min Floating Spread Test - With Libor Floors',weight_col] = \
        weighted_average(model_df,cols=[weight_col,'All In Rate'])*100
    
    Port_stats_df.loc['Max Moodys Rating Factor Test (NEW WARF)',weight_col] = \
        weighted_average(model_df,cols=[weight_col,'Adj. WARF NEW'])
        #moodys_adjusted_warf(model_df)  #Adj. WARF NEW
     
    Port_stats_df.loc['Max Moodys Rating Factor Test (Orig WARF)',weight_col] = \
        weighted_average(model_df,cols=[weight_col,'WARF'])
        #moodys_adjusted_warf_old(model_df)
    
    
        
    Port_stats_df.loc['Min Moodys Recovery Rate Test',weight_col] = \
        weighted_average(model_df,cols=[weight_col,'Moodys Recovery Rate'])*100    
    
    Port_stats_df.loc['Min S&P Recovery Rate Class A-1a',weight_col] = \
            weighted_average(model_df,cols=[weight_col,'S&P Recovery Rate (AAA)'])*100
        #weighted_average(model_df,cols=[weight_col,'S&P Recovery Rate (AAA)'])*100
    
    Port_stats_df.loc['Moodys Diversity Test',weight_col] = diversity_score(model_df, weight_col)
    
    ######   Was Total?
    #Port_stats_df.loc['WAP',weight_col] = \
    #    weighted_average(model_df,cols=[weight_col,'Blended Price'])
    
    # it needs a different amort table for each CLO, so this is place holder
    Port_stats_df.loc['Weighted Average Life Test',weight_col] = \
            Weighted_Average_Life(clo_df,weight_col)
    
    Port_stats_df.loc['Percent CCC & lower',weight_col] = percentage_C(model_df, weight_col)*100
    
    Port_stats_df.loc['Percent 2nd Lien',weight_col] = percentage_SecondLien(model_df, weight_col)*100
    
    #Port_stats_df.loc['Percent Sub80',weight_col] = percentage_SubEighty(model_df, weight_col)*100
    
    #Port_stats_df.loc['Percent Sub90',weight_col] = percentage_SubNinety(model_df, weight_col)*100
    
    #Port_stats_df.loc['Percent CovLite',weight_col] = percentage_CovLite(model_df, weight_col)*100
    Port_stats_df.loc['Total Portfolio Par (excl. Defaults)',weight_col] = model_df[weight_col].sum()

    
    if format_output:
        Port_stats_df.loc['Min Floating Spread Test - no Libor Floors'] = \
            Port_stats_df.loc['Min Floating Spread Test - no Libor Floors'].apply('{:.2f}%'.format)
        Port_stats_df.loc['Min Floating Spread Test - With Libor Floors'] = \
            Port_stats_df.loc['Min Floating Spread Test - With Libor Floors'].apply('{:.2f}%'.format)
        Port_stats_df.loc['Max Moodys Rating Factor Test (NEW WARF)'] = \
            Port_stats_df.loc['Max Moodys Rating Factor Test (NEW WARF)'].apply('{:.0f}'.format)
        Port_stats_df.loc['Max Moodys Rating Factor Test (Orig WARF)'] = \
            Port_stats_df.loc['Max Moodys Rating Factor Test (Orig WARF)'].apply('{:.0f}'.format)
        Port_stats_df.loc['Min Moodys Recovery Rate Test'] = \
            Port_stats_df.loc['Min Moodys Recovery Rate Test'].apply('{:.1f}%'.format)
        Port_stats_df.loc['Min S&P Recovery Rate Class A-1a'] = \
            Port_stats_df.loc['Min S&P Recovery Rate Class A-1a'].apply('{:.1f}%'.format)
        Port_stats_df.loc['Moodys Diversity Test'] = \
            Port_stats_df.loc['Moodys Diversity Test'].apply('{:.0f}'.format)
        #Port_stats_df.loc['WAP'] = \
        #    Port_stats_df.loc['WAP'].apply('${:.2f}'.format)
        Port_stats_df.loc['Weighted Average Life Test'] = \
            Port_stats_df.loc['Weighted Average Life Test'].apply('{:.2f}'.format)
        Port_stats_df.loc['Percent CCC & lower'] = \
            Port_stats_df.loc['Percent CCC & lower'].apply('{:.1f}%'.format)
        Port_stats_df.loc['Percent 2nd Lien'] = \
            Port_stats_df.loc['Percent 2nd Lien'].apply('{:.1f}%'.format)
        #Port_stats_df.loc['Percent Sub80'] = \
        #    Port_stats_df.loc['Percent Sub80'].apply('{:.1f}%'.format)
        #Port_stats_df.loc['Percent Sub90'] = \
        #    Port_stats_df.loc['Percent Sub90'].apply('{:.1f}%'.format)
        #Port_stats_df.loc['Percent CovLite'] = \
        #    Port_stats_df.loc['Percent CovLite'].apply('{:.1f}%'.format)
        Port_stats_df.loc['Total Portfolio Par (excl. Defaults)'] = \
            Port_stats_df.loc['Total Portfolio Par (excl. Defaults)'].apply('{:,.0f}'.format)
        
  
    #Port_stats_df.loc['Pot Par B/L Sale',weight_col] = model_df['Par_Build_Loss_Sale'].sum()
    #Port_stats_df.loc['Pot Par B/L Sale'] = \
    #    Port_stats_df.loc['Pot Par B/L Sale'].apply('{:,.0f}'.format)    
    
    #Port_stats_df.loc['Pot Par B/L Buy',weight_col] = model_df['Par_Build_Loss_Buy'].sum()
    #Port_stats_df.loc['Pot Par B/L Buy'] = \
    #    Port_stats_df.loc['Pot Par B/L Buy'].apply('{:,.0f}'.format)    
    
    #Port_stats_df.loc['Pot Par B/L Total',weight_col] = model_df['Total_Par_Build_Loss'].sum()
    #Port_stats_df.loc['Pot Par B/L Total'] = \
    #    Port_stats_df.loc['Pot Par B/L Total'].apply('{:,.0f}'.format)    

    
#    Port_stats_df.loc['Total Portfolio Par',weight_col] = model_df['Total'].sum()
#    Port_stats_df.loc['Total Portfolio Par'] = \
#        Port_stats_df.loc['Total Portfolio Par'].apply('{:,.0f}'.format)
    
    # current portfolio is Quantity + Add'l Amount (manual) TBA later
#    Port_stats_df.loc['Current Portfolio',weight_col] = model_df[['Addtl Purchase Amt','Current Portfolio']].sum(axis=1).sum()
#    Port_stats_df.loc['Current Portfolio'] = \
#        Port_stats_df.loc['Current Portfolio'].apply('{:,.0f}'.format)
    
    return Port_stats_df

In [55]:
def DataLabelMap(df):
    datalabelmap = {'Parent Company': 'Parent Company',
        'Issuer': 'Issuer',
        'Asset': 'Asset',
        'CUSIP': 'CUSIP',
        'ISIN': 'ISIN',
        'Asset Type': 'Asset Type',
        'Analyst': 'Analyst',
        'Floating Spread': 'Spread',
        'Floating Spread Floor': 'Floor',
        'All In Rate': 'All In Rate',
        'Maturity Date': 'Maturity Date',
        'Mark Price': 'Mark Price',
        'Adjusted Moodys Rating': 'Adjusted Moodys Rating',
        'WARF': 'WARF',
        'MDPR': 'MDPR',
        'Moody\'s CFR': 'Moodys CFR',
        'Moody\'s Facility Rating': 'Moodys Facility Rating',
        'Moody\'s Issuer Outlook': 'Moodys Issuer Outlook',
        'Moody\'s Issuer Watch': 'Moodys Issuer Watch',
        'Moodys Recovery Rate': 'Moodys Recovery Rate',
        'S&P Issuer Rating': 'S&P Issuer Rating',
        'S&P Facility Rating': 'S&P Facility Rating',
        'S&P Issuer Outlook': 'S&P Issuer Outlook',
        'S&P Issuer Watch': 'S&P Issuer Watch',
        'S&P Recovery': 'S&P Recovery',
        'Agent Bank': 'Agent Bank',
        'OCP CLO 2013-4, Ltd.': 'CLO 2013-4',
        'OCP CLO 2014-5, Ltd.': 'CLO 2014-5',
        'OCP CLO 2014-6, Ltd.': 'CLO 2014-6',
        'OCP CLO 2014-7, Ltd.': 'CLO 2014-7',
        'OCP CLO 2020-8R, Ltd.': 'CLO 2020-8R',
        'OCP CLO 2015-9, Ltd.': 'CLO 2015-9',
        'OCP CLO 2015-10, Ltd.': 'CLO 2015-10',
        'OCP CLO 2016-11, Ltd.': 'CLO 2016-11',
        'OCP CLO 2016-12, Ltd.': 'CLO 2016-12',
        'OCP CLO 2017-13, Ltd.': 'CLO 2017-13',
        'OCP CLO 2017-14, Ltd.': 'CLO 2017-14',
        'OCP CLO 2018-15, Ltd.': 'CLO 2018-15',
        'OCP CLO 2019-16, Ltd.': 'CLO 2019-16',
        'OCP CLO 2019-17, Ltd': 'CLO 2019-17',
        'OCP CLO 2020-18, Ltd.': 'CLO 2020-18',
        'OCP CLO 2020-19, Ltd.': 'CLO 2020-19',
        'OCP CLO 2020-20, Ltd.': 'CLO 2020-20',
        'OCP CLO 2021-21 Ltd.': 'CLO 2021-21',
        'Current Global Amount Outstanding': 'Current Global Amount Outstanding',
        'Moody\'s Industry': 'Moodys Industry',
        'S&P Industry': 'S&P Industry',
        'Lien Type': 'Lien Type',
        'Issuer Country': 'Issuer Country'}
    df.columns = df.columns.str.strip()  # removes leading/trailing whitespace
    df.rename(columns=datalabelmap,inplace=True)
    return df



def get_master_position_report(filepath):
    master_df = pd.read_excel(filepath,header=0)
    master_df = master_df.loc[:,~master_df.columns.str.match("Unnamed")]
    master_df.rename(columns={'LoanX ID':'LXID'},inplace=True)
    master_df.set_index('LXID', inplace=True)
    master_df = DataLabelMap(master_df)
    master_df = add_derived_features(master_df,libor=0.0020)
    return master_df

In [56]:
def diversity_score(model_df, weight_col='Par_no_default'):
    """
    This function calculates the Moody's Industry Diversity Score for the CLO
    
    Arg in:
        model_df: the input data frame (from the MASTER table d/l'd from BMS)
        #ind_avg_eu: Moody's discrete lookup table that maps AIEUS to IDS, need to be sorted
    Arg out:
        dscore: the scalar measure of the IDS
    """
    # calling this each time (like in MC Diversity)
    # makes the function unbearably slow. I tried setting it
    # as a global variable but that didn't solve the issue.
    # I need to look into a solution to set something up 
    # permanently in memory after the first call (thought global would do it)
    # ind_avg_eu = get_ind_avg_eu_table(filepath,sheet='Diversity')
    # thus I've hard-coded it, but if this ever changes from Moody's
    # we need to update here.
    ind_avg_eu = pd.DataFrame([[ 0.    ,  0.    ], [ 0.05  ,  0.1   ], [ 0.15  ,  0.2   ],
       [ 0.25  ,  0.3   ], [ 0.35  ,  0.4   ], [ 0.45  ,  0.5   ], [ 0.55  ,  0.6   ], 
       [ 0.65  ,  0.7   ], [ 0.75  ,  0.8   ], [ 0.85  ,  0.9   ], [ 0.95  ,  1.    ],
       [ 1.05  ,  1.05  ], [ 1.15  ,  1.1   ], [ 1.25  ,  1.15  ], [ 1.35  ,  1.2   ],
       [ 1.45  ,  1.25  ], [ 1.55  ,  1.3   ], [ 1.65  ,  1.35  ], [ 1.75  ,  1.4   ],
       [ 1.85  ,  1.45  ], [ 1.95  ,  1.5   ], [ 2.05  ,  1.55  ], [ 2.15  ,  1.6   ],
       [ 2.25  ,  1.65  ], [ 2.35  ,  1.7   ], [ 2.45  ,  1.75  ], [ 2.55  ,  1.8   ],
       [ 2.65  ,  1.85  ], [ 2.75  ,  1.9   ], [ 2.85  ,  1.95  ], [ 2.95  ,  2.    ],
       [ 3.05  ,  2.0333], [ 3.15  ,  2.0667], [ 3.25  ,  2.1   ], [ 3.35  ,  2.1333],
       [ 3.45  ,  2.1667], [ 3.55  ,  2.2   ], [ 3.65  ,  2.2333], [ 3.75  ,  2.2667],
       [ 3.85  ,  2.3   ], [ 3.95  ,  2.3333], [ 4.05  ,  2.3667], [ 4.15  ,  2.4   ],
       [ 4.25  ,  2.4333], [ 4.35  ,  2.4667], [ 4.45  ,  2.5   ], [ 4.55  ,  2.5333],
       [ 4.65  ,  2.5667], [ 4.75  ,  2.6   ], [ 4.85  ,  2.6333], [ 4.95  ,  2.6667],
       [ 5.05  ,  2.7   ], [ 5.15  ,  2.7333], [ 5.25  ,  2.7667], [ 5.35  ,  2.8   ],
       [ 5.45  ,  2.8333], [ 5.55  ,  2.8667], [ 5.65  ,  2.9   ], [ 5.75  ,  2.9333],
       [ 5.85  ,  2.9667], [ 5.95  ,  3.    ], [ 6.05  ,  3.025 ], [ 6.15  ,  3.05  ],
       [ 6.25  ,  3.075 ], [ 6.35  ,  3.1   ], [ 6.45  ,  3.125 ], [ 6.55  ,  3.15  ],
       [ 6.65  ,  3.175 ], [ 6.75  ,  3.2   ], [ 6.85  ,  3.225 ], [ 6.95  ,  3.25  ],
       [ 7.05  ,  3.275 ], [ 7.15  ,  3.3   ], [ 7.25  ,  3.325 ], [ 7.35  ,  3.35  ],
       [ 7.45  ,  3.375 ], [ 7.55  ,  3.4   ], [ 7.65  ,  3.425 ], [ 7.75  ,  3.45  ],
       [ 7.85  ,  3.475 ], [ 7.95  ,  3.5   ], [ 8.05  ,  3.525 ], [ 8.15  ,  3.55  ],
       [ 8.25  ,  3.575 ], [ 8.35  ,  3.6   ], [ 8.45  ,  3.625 ], [ 8.55  ,  3.65  ],
       [ 8.65  ,  3.675 ], [ 8.75  ,  3.7   ], [ 8.85  ,  3.725 ], [ 8.95  ,  3.75  ],
       [ 9.05  ,  3.775 ], [ 9.15  ,  3.8   ], [ 9.25  ,  3.825 ], [ 9.35  ,  3.85  ],
       [ 9.45  ,  3.875 ], [ 9.55  ,  3.9   ], [ 9.65  ,  3.925 ], [ 9.75  ,  3.95  ],
       [ 9.85  ,  3.975 ], [ 9.95  ,  4.    ], [10.05  ,  4.01  ], [10.15  ,  4.02  ],
       [10.25  ,  4.03  ], [10.35  ,  4.04  ], [10.45  ,  4.05  ], [10.55  ,  4.06  ],
       [10.65  ,  4.07  ], [10.75  ,  4.08  ], [10.85  ,  4.09  ], [10.95  ,  4.1   ],
       [11.05  ,  4.11  ], [11.15  ,  4.12  ], [11.25  ,  4.13  ], [11.35  ,  4.14  ],
       [11.45  ,  4.15  ], [11.55  ,  4.16  ], [11.65  ,  4.17  ], [11.75  ,  4.18  ],
       [11.85  ,  4.19  ], [11.95  ,  4.2   ], [12.05  ,  4.21  ], [12.15  ,  4.22  ],
       [12.25  ,  4.23  ], [12.35  ,  4.24  ], [12.45  ,  4.25  ], [12.55  ,  4.26  ],
       [12.65  ,  4.27  ], [12.75  ,  4.28  ], [12.85  ,  4.29  ], [12.95  ,  4.3   ],
       [13.05  ,  4.31  ], [13.15  ,  4.32  ], [13.25  ,  4.33  ], [13.35  ,  4.34  ],
       [13.45  ,  4.35  ], [13.55  ,  4.36  ], [13.65  ,  4.37  ], [13.75  ,  4.38  ],
       [13.85  ,  4.39  ], [13.95  ,  4.4   ], [14.05  ,  4.41  ], [14.15  ,  4.42  ],
       [14.25  ,  4.43  ], [14.35  ,  4.44  ], [14.45  ,  4.45  ], [14.55  ,  4.46  ],
       [14.65  ,  4.47  ], [14.75  ,  4.48  ], [14.85  ,  4.49  ], [14.95  ,  4.5   ],
       [15.05  ,  4.51  ], [15.15  ,  4.52  ], [15.25  ,  4.53  ], [15.35  ,  4.54  ],
       [15.45  ,  4.55  ], [15.55  ,  4.56  ], [15.65  ,  4.57  ], [15.75  ,  4.58  ],
       [15.85  ,  4.59  ], [15.95  ,  4.6   ], [16.05  ,  4.61  ], [16.15  ,  4.62  ],
       [16.25  ,  4.63  ], [16.35  ,  4.64  ], [16.45  ,  4.65  ], [16.55  ,  4.66  ],
       [16.65  ,  4.67  ], [16.75  ,  4.68  ], [16.85  ,  4.69  ], [16.95  ,  4.7   ],
       [17.05  ,  4.71  ], [17.15  ,  4.72  ], [17.25  ,  4.73  ], [17.35  ,  4.74  ],
       [17.45  ,  4.75  ], [17.55  ,  4.76  ], [17.65  ,  4.77  ], [17.75  ,  4.78  ],
       [17.85  ,  4.79  ], [17.95  ,  4.8   ], [18.05  ,  4.81  ], [18.15  ,  4.82  ],
       [18.25  ,  4.83  ], [18.35  ,  4.84  ], [18.45  ,  4.85  ], [18.55  ,  4.86  ],
       [18.65  ,  4.87  ], [18.75  ,  4.88  ], [18.85  ,  4.89  ], [18.95  ,  4.9   ],
       [19.05  ,  4.91  ], [19.15  ,  4.92  ], [19.25  ,  4.93  ], [19.35  ,  4.94  ], [19.45  ,  4.95  ],
       [19.55  ,  4.96  ], [19.65  ,  4.97  ], [19.75  ,  4.98  ], [19.85  ,  4.99  ], [19.95  ,  5.    ]], 
                              columns = ['Aggregate\nIndustry\nEquivalent\nUnit Score', 'Industry\nDiversity\nScore'])    
    #first create the Par amount filtering out defaults
    #model_df['Par_no_default'] = model_df['Total']
    #model_df.loc[model_df['Default']=='Y','Par_no_default'] = 0
    
    # 6/14/2021 added this...it is correct
    # filters out the zero names from the average Par Amount (in the count)
    mask = abs(model_df[weight_col]) > 0
    
    
    div_df = model_df.loc[mask,['Parent Company','Moodys Industry',weight_col]].copy()
    div_df.sort_values(by='Moodys Industry',inplace=True)

    # this keeps the industry, but groups on parent company for multiple loans
    # shouldn't this be Obligor for Diversity field and not Parent Company?
    # they are the same in our spreadsheet, but maybe need to be careful
    test = div_df.groupby(by=['Parent Company','Moodys Industry']).sum()
    avg_par_amt = test.sum()/test.count()   
    
    # create the EU score for each parent
    # Lesser of 1 and Issuer Par Amount for such issuer divided by the Average Par Amount.
    test['EU'] = test[[weight_col]]/test[[weight_col]].mean()
    test.loc[test['EU']>1,'EU']=1
    
    # groupby Industry for the Ind Div Score
    IDS = test.groupby(by=['Moodys Industry']).sum()

    # this is like vlookup(..,TRUE) where the nearest match on merge is used, direction controls how
    # backward is the lesser if EU falls between AIEUS marks
    df_merged = pd.merge_asof(IDS.sort_values('EU'), ind_avg_eu, left_on='EU', 
                          right_on='Aggregate\nIndustry\nEquivalent\nUnit Score', direction='backward', suffixes=['', '_2'])
    dscore = df_merged['Industry\nDiversity\nScore'].sum()
    return dscore

In [57]:
def weighted_average(model_df,cols):
    """
    Calculates the weighted average variable
    Args in:
        model_df
        cols: must be a list of two elements with the first element
                the weighting elements and the second the statistic to be weighted
                i.e. ['weight','stat']
    Arg out: 
        wa: scalar
    """
    #wa = model_df[cols].apply(lambda x: (x[0]*x[1]).sum()/x[0].sum())
    wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols[0]].sum()
    return wa

In [58]:
def SP_CDO_Monitor_Test(clo_df,col='CLO 2020-20'):
    #C0 = 0.185893082941552
    #C1 = 3.70021989750327
    #C2 = 0.881097473356921
    C0=0.109706
    C1=4.140929
    C2=1.064031
    
    # 'S&P Issuer Rating'
    #clo_df.drop(columns=['S&P CLO Specified Assets'],inplace=True)
    clo_df['S&P CLO Specified Assets'] = clo_df[col].values
    mask = (clo_df['S&P Issuer Rating']=='CC') | \
           (clo_df['S&P Issuer Rating']=='SD') | \
           (clo_df['S&P Issuer Rating']=='NR') | \
           (clo_df['S&P Issuer Rating']=='D')
    clo_df.loc[mask,'S&P CLO Specified Assets'] = 0   # what about .isna()?
   
    # Target Par Amount
    # this is a quick estimate until we have a default parameter set for each
    OP = (round((clo_df[col].sum()/1e8))*100)*1e6

    
    # Agg Principal Balance (excl < CCC-)
    NP = clo_df['S&P CLO Specified Assets'].sum()
    
    print("OP: ", OP, ", NP: ", NP)
    # Weighted Average Spread
    SPWAS = weighted_average(clo_df,cols=[col,'Spread'])
    
    # Weighted Average Recovery Rate
    # uses par no default basically
    SPWARR = weighted_average(clo_df,cols=[col,'S&P Recovery Rate (AAA)'])
    
    # Weighted Average Rating Factor
    # uses specified assets
    SPWARF = weighted_average(clo_df,cols=['S&P CLO Specified Assets','S&P Global Ratings Factor'])
    
    clo_df['Absolute Deviation'] = abs(clo_df['S&P Global Ratings Factor']-SPWARF)
    # Default Rate Dispersion
    # weighted average of |Global Rating Factor - WARF| (excl < CCC-)
    # use specified assets
    DRD = weighted_average(clo_df,cols=['S&P CLO Specified Assets','Absolute Deviation'])
    
    #print("SPWAS: ", SPWAS, ", SPWARR: ", SPWARR, ', SPWARF: ',SPWARF, ", DRD: ", DRD)
    
    # Obligor Diversity Measure
    vec = clo_df[[col,'Parent Company']].groupby('Parent Company').sum().values/NP
    ODM = 1/vec.T.dot(vec)
    
    # Industry Diversity Measure
    vec = clo_df[[col,'S&P Industry']].groupby('S&P Industry').sum().values/NP
    IDM = 1/vec.T.dot(vec)
    
    # Regional Diversity Measure
    # Annex D : S&P CDO Evaluator Country Codes, Regions and Recovery Groups
    vec = clo_df[[col,'Issuer Country']].groupby('Issuer Country').sum().values/NP
    RDM = 1/vec.T.dot(vec)
    
    # Weighted Average Life
    WAL = Weighted_Average_Life(clo_df,col)
    
    #print("ODM: ", ODM, ", IDM: ", IDM, ', RDM: ',RDM, ", WAL: ", WAL)
    
    # S&P CDO Monitor BDR
    BDR = C0 + (C1 * SPWAS) + (C2 * SPWARR)
    
    # S&P CDO Monitor Adjusted BDR
    AdjBDR = BDR * (OP/NP) - (NP - OP)/(NP *(1-SPWARR))
    
    # S&P CDO Monitor SDR
    # 0.247621 + (SPWARF/9162.65) – (DRD/16757.2) – (ODM/7677.8) – (IDM/2177.56) – 
    # (RDM/34.0948) + (WAL/27.3896)
    SDR = 0.247621 + (SPWARF/9162.65) - (DRD/16757.2) - (ODM/7677.8) - (IDM/2177.56) - (RDM/34.0948) \
        + (WAL/27.3896)
    
    print('Pass' if AdjBDR >= SDR else 'Fail')
    
    SP_CDO_mon_df = pd.DataFrame(np.nan,index=['S&P Weighted Average Rating Factor (SP WARF)',
        'Default Rate Dispersion (DRD)',
        'Obligor Diversity Measure (ODM)',
        'Industry Diversity Measure (IDM)',
        'Regional Diversity Measure (RDM)',
        'Weighted Average Life (WAL)',
        'Break-Even Default Rate (BDR)',
        'Adjusted Break-Even Default Rate (Adj BDR)',
        'Scenario Default Rate (SDR)'],columns = [col])

    SP_CDO_mon_df.loc['S&P Weighted Average Rating Factor (SP WARF)',col] =  SPWARF
    SP_CDO_mon_df.loc['Default Rate Dispersion (DRD)',col] = DRD
    SP_CDO_mon_df.loc['Obligor Diversity Measure (ODM)',col] = ODM
    SP_CDO_mon_df.loc['Industry Diversity Measure (IDM)',col] = IDM
    SP_CDO_mon_df.loc['Regional Diversity Measure (RDM)',col] = RDM
    SP_CDO_mon_df.loc['Weighted Average Life (WAL)',col] = WAL
    SP_CDO_mon_df.loc['Break-Even Default Rate (BDR)',col] = BDR
    SP_CDO_mon_df.loc['Adjusted Break-Even Default Rate (Adj BDR)',col] = AdjBDR
    SP_CDO_mon_df.loc['Scenario Default Rate (SDR)',col] = SDR
    
    return SP_CDO_mon_df

In [59]:
def moodys_adjusted_warf(df):
    """
    This function creates the new Moody's Ratings Factor based 
    on the old Moody's rating.
    
    Arg in:
        df: the input data frame (from the MASTER table d/l'd from BMS)     
    """
    
    # moodys_score: dataframe with alphanumeric rating to numeric map (1 to 1 map; linear)
    moodys_score = pd.DataFrame([[ 'Aaa',1],['Aa1',2],['Aa2',3],['Aa3',4],
              ['A1',5],['A2',6],['A3',7],['Baa1',8],['Baa2',9],
              ['Baa3',10],['Ba1',11],['Ba2',12],['Ba3',13],
              ['B1',14],['B2',15],['B3',16],['Caa1',17],
              ['Caa2',18],['Caa3',19],['Ca',20],['C',21]],columns=['Moodys','Score'])
    
    # moodys_rfTable: dataframe with alphanumeric rating to new WARF numeric (1 to 1 map; 1 to 1000 values)
    moodys_rfTable = pd.DataFrame([[ 'Aaa',1],['Aa1',10],['Aa2',20],['Aa3',40],
              ['A1',70],['A2',120],['A3',180],['Baa1',260],['Baa2',360],
              ['Baa3',610],['Ba1',940],['Ba2',1350],['Ba3',1766],
              ['B1',2220],['B2',2720],['B3',3490],['Caa1',4770],
              ['Caa2',6500],['Caa3',8070],['Ca',10000],['C',10000],
              ['NR',10000],['W/D',10000]],columns=['Moodys Rating Factor Table','RF'])
    
    score = df['Moodys CFR'].map(dict(moodys_score[['Moodys','Score']].values))
    updown = df['Moodys Issuer Watch'].\
        apply(lambda x: -1 if x == 'Possible Upgrade' else 1 if x == 'Possible Downgrade' else 0)
    aScore = score + updown
    Adjusted_CFR_for_WARF = aScore.map(dict(moodys_score[['Score','Moodys']].values))
    # I keep the same column name as Jeff to make it easier to double check values
    df['Adj. WARF NEW'] = Adjusted_CFR_for_WARF.map(dict(moodys_rfTable[['Moodys Rating Factor Table','RF']].values))
    return df

def moodys_adjusted_warf_old(df):
    """
    This function creates the new Moody's Ratings Factor based 
    on the old Moody's rating. 
    
    !!! This needs to have the old WARF logic added, just a place holder ATM !!!
    
    Arg in:
        df: the input data frame (from the MASTER table d/l'd from BMS)     
    """
    
    # moodys_score: dataframe with alphanumeric rating to numeric map (1 to 1 map; linear)
    moodys_score = pd.DataFrame([[ 'Aaa',1],['Aa1',2],['Aa2',3],['Aa3',4],
              ['A1',5],['A2',6],['A3',7],['Baa1',8],['Baa2',9],
              ['Baa3',10],['Ba1',11],['Ba2',12],['Ba3',13],
              ['B1',14],['B2',15],['B3',16],['Caa1',17],
              ['Caa2',18],['Caa3',19],['Ca',20],['C',21]],columns=['Moodys','Score'])
    
    # moodys_rfTable: dataframe with alphanumeric rating to new WARF numeric (1 to 1 map; 1 to 1000 values)
    moodys_rfTable = pd.DataFrame([[ 'Aaa',1],['Aa1',10],['Aa2',20],['Aa3',40],
              ['A1',70],['A2',120],['A3',180],['Baa1',260],['Baa2',360],
              ['Baa3',610],['Ba1',940],['Ba2',1350],['Ba3',1766],
              ['B1',2220],['B2',2720],['B3',3490],['Caa1',4770],
              ['Caa2',6500],['Caa3',8070],['Ca',10000],['C',10000],
              ['NR',10000],['W/D',10000]],columns=['Moodys Rating Factor Table','RF'])
    
    score = df['Moodys CFR'].map(dict(moodys_score[['Moodys','Score']].values))
    updown = df['Moodys Issuer Watch'].\
        apply(lambda x: -1 if x == 'Possible Upgrade' else 1 if x == 'Possible Downgrade' else 0)
    aScore = score + updown
    Adjusted_CFR_for_WARF = aScore.map(dict(moodys_score[['Score','Moodys']].values))
    # I keep the same column name as Jeff to make it easier to double check values
    df['Adj. WARF NEW'] = Adjusted_CFR_for_WARF.map(dict(moodys_rfTable[['Moodys Rating Factor Table','RF']].values))
    return df

def sp_grf(df):
    sp_grf = pd.DataFrame([['AAA',13.51],['AA+',26.75],['AA',46.36],['AA-',63.90],
                            ['A+',99.50],['A',146.35],['A-',199.83],['BBB+',271.01],
                            ['BBB',361.17],['BBB-',540.42],['BB+',784.92],['BB',1233.63],
                            ['BB-',1565.44],['B+',1982.00],['B',2859.50],['B-',3610.11],
                            ['CCC+',4641.40],['CCC',5293.00],['CCC-',5751.00],['CC',10000.00],
                            ['SD',10000.00],['D',10000.00]],columns = ['S&P Issuer Rating','S&P Global Ratings Factor'])
    df['S&P Global Ratings Factor'] = df['S&P Issuer Rating'].map(dict(sp_grf[['S&P Issuer Rating','S&P Global Ratings Factor']].values))
    return df

def add_derived_features(df,libor=0.0020):
    df = moodys_adjusted_warf(df)   # 'Adj. WARF NEW'
    df = sp_recovery_rate(df)       # 'S&P Recovery Rate (AAA)'
    df = sp_grf(df)                 # 'S&P Global Ratings Factor'
    df = Effective_Spread(df,libor=0.0020)  # Calc the EffSpread = Spread + max(floor-libor,0)

    return df


def sp_recovery_rate(model_df):
    """
    This function get the S&P recovery rate as a percent. If it doesn't exist
    in the master field, it will look up in the appropriate first and second 
    lien tables, if not, will look up the bond table.
    
    Arg in:
        model_df: the input data frame (from the MASTER table d/l'd from BMS)
        lien: a DF table with the RR's for first and second lien by country
        new_rr: a df mapping of the old notation for RR to a new RR in percentage
        bond_table: split out of a table for RR for bonds
    Arg out:
        model_df with inserted new column 'S&P Recovery Rate (AAA)'
    """
    new_rr_map = {'1+(100)': 0.75,
                '1(95%)': 0.70,
                '1(90%)': 0.65,
                '2(85%)': 0.625,
                '2(80%)': 0.60,
                '2(75%)': 0.55,
                '2(70%)': 0.5,
                '3(65%)': 0.45,
                '3(60%)': 0.4,
                '3(55%)': 0.35,
                '3(50%)': 0.3,
                '4(45%)': 0.285,
                '4(40%)': 0.27,
                '4(35%)': 0.235,
                '4(30%)': 0.20,
                '5(25%)': 0.175,
                '5(20%)': 0.15,
                '5(15%)': 0.10,
                '5(10%)': 0.05,
                '6(5%)': 0.035,
                '6(0%)': 0.02,
                '3H': 0.40,
                '1': 0.65}
    
    LienOne_map = {'AU':0.50,'AT':0.50,'BE':0.50,
            'CA':0.50,'DK':0.50,'FI':0.50,'FR':0.50,
            'DE':0.50,'HK':0.50,'IE':0.50,'IS':0.50,
            'JP':0.50,'LU':0.50,'NL':0.50,'NO':0.50,
            'PO':0.50,'PT':0.50,'SG':0.50,'ES':0.50,
            'SE':0.50,'CH':0.50,'GB':0.50,'US':0.50,
            'BR':0.39,'CZ':0.39,'GR':0.39,'IT':0.39,
            'MX':0.39,'ZA':0.39,'TR':0.39,'UA':0.39}
    LienTwo_map = {'AU':0.18,'AT':0.18,'BE':0.18,
            'CA':0.18,'DK':0.18,'FI':0.18,'FR':0.18,
            'DE':0.18,'HK':0.18,'IE':0.18,'IS':0.18,
            'JP':0.18,'LU':0.18,'NL':0.18,'NO':0.18,
            'PO':0.18,'PT':0.18,'SG':0.18,'ES':0.18,
            'SE':0.18,'CH':0.18,'GB':0.18,'US':0.18,
            'BR':0.13,'CZ':0.13,'GR':0.13,'IT':0.13,
            'MX':0.13,'ZA':0.13,'TR':0.13,'UA':0.13}
    
    bond_map = {'US':0.41}
 
        
    # if it the Recovery rate exists lookup in AAA table
    model_df['S&P Recovery Rate (AAA)'] = model_df['S&P Recovery'].map(new_rr_map)
        #map(dict(new_rr[['S&P Recovery Rating\nand Recovery\nIndicator of\nCollateral Obligations','“AAA”']].values))
    
    # doesn't exist, but first lien, use first lien table
    model_df.loc[pd.isna(model_df['S&P Recovery']) & (model_df['Lien Type']== 'First Lien'),'S&P Recovery Rate (AAA)'] =\
        model_df.loc[pd.isna(model_df['S&P Recovery']) & (model_df['Lien Type']== 'First Lien'),'Issuer Country'].\
        map(LienOne_map)
        #map(dict(lien[['Country Abv','RR']].values))
    
    
    # doesn't exist, but 2nd lien, use 2nd lien table
    model_df.loc[pd.isna(model_df['S&P Recovery']) & (model_df['Lien Type']== 'Second Lien'),'S&P Recovery Rate (AAA)'] = \
        model_df.loc[pd.isna(model_df['S&P Recovery']) & (model_df['Lien Type']== 'Second Lien'),'Issuer Country'].\
        map(LienTwo_map)
        #map(dict(lien[['Country Abv','RR.2nd']].values))
    
    # the bonds
    model_df.loc[pd.isna(model_df['S&P Recovery']) & pd.isna(model_df['Lien Type']),'S&P Recovery Rate (AAA)'] = \
        model_df.loc[pd.isna(model_df['S&P Recovery']) & pd.isna(model_df['Lien Type']),'Issuer Country'].\
        map(bond_map)
        #map(dict(bond_table[['Country Abv.1','RR.1']].values))

    return model_df

In [60]:
def Specified_Assets(df,col):
    df['S&P CLO Specified Assets'] = df[col].values
    mask = (df['S&P Issuer Rating']=='CC') | \
           (df['S&P Issuer Rating']=='SD') | \
           (df['S&P Issuer Rating']=='NR') | \
           (df['S&P Issuer Rating']=='D')
    df.loc[mask,'S&P CLO Specified Assets'] = 0   # what about .isna()?
    return df
    

def Weighted_Average_Life(clo_df,col):
   
    clo_df['Average Life']  = ((clo_df['Maturity Date'] - pd.Timestamp.today()).dt.days/365).round(2)
    clo_df = Specified_Assets(clo_df,col)
    
    WAL_stat = weighted_average(clo_df,cols=['S&P CLO Specified Assets','Average Life'])
    
    return WAL_stat

def Weighted_Average_Spread(clo_df,col,libor=.002):
    """
    Per the Virtus Compliance Reports, the Indenture and the Cheat-sheet-Spreadsheet
    the WAS is based on the 'Effective Spread' which is neither the Floating Spread,
    nor the All-in-Rate in the files, but rather derived from the Floating Spread and the 
    Floor vis a vis LIBOR.  It is possible that this needs further refinement, but this
    gets us much closer to the target.
    
    Specified Assets uses all Principal Balance/(Par) less CC/Ca and lower rated deals.
    WAS also uses only Floating Rate Loans, not Fixed Rate Bonds, which are included in the WAC
    
    Args in:
        clo_df:  the dataframe, df
        col:   Which column represents the CLO in question, string
        libor:  latest estimate of LIBOR, float
    Arg out:
        WAS_stat:  The effective WAS, float
    """
   
    clo_df = Specified_Assets(clo_df,col)   # This should be a view, not a copy:  sub CCC excluded
    mask = (clo_df['Asset Type']!='Bond') & (clo_df[col] > 0) & (~clo_df['Spread'].isna())  # Bonds are excluded too
    
    WAS_stat = weighted_average(clo_df.loc[mask],cols=['S&P CLO Specified Assets','Effective Spread'])
    
    return WAS_stat

def Effective_Spread(df,libor=0.0020):
    """
    The default LIBOR is 20 bps, which was relavant on 6/4/21 
    (to be fair, on 6/4 the 3m LIBOR = 12.7bps, but it was used as
    20bps many places in the examples, so that is the default. 
    Today it is closer to 15bps 6/25/21)
    I'd call the bloomberg api, but then it wouldn't work broadly
    """
    
    def Libor_floor(df,libor = 0.0020):
        df['Libor Floor'] = df['Floor']-libor
        df.loc[df['Libor Floor']<0,'Libor Floor'] = 0
        return df

    df['Effective Spread'] = df['Spread'] + Libor_floor(df,libor = 0.0020)['Libor Floor']

    
    return df


def Weighted_Average_Coupon(clo_df,col):
    """
    The Test uses WAC + Excess WAFS (currently ~11%)
    The WAC is probably closer to the WAS
    """
    mask = (clo_df['Asset Type']=='Bond')
    WAC_stat = weighted_average(clo_df.loc[mask],['CLO 2020-20','All In Rate'])
    
    return WAC_stat

def Excess_Weighted_Average_FloatSpread(clo_df,col,minFS):
    """
    This doesn't seem correct.  Is it units or fail?
    """
    mask = (clo_df['Asset Type']!='Bond') & (clo_df['CLO 2020-20'] > 0) & (~clo_df['Spread'].isna())
    float_Par = clo_df.loc[mask,col].sum()
    
    mask = (clo_df['Asset Type']=='Bond')
    fixed_Par = clo_df.loc[mask,col].sum()
    
    EWAFS = (Weighted_Average_Spread(clo_df,col)-minFS)*float_Par/fixed_Par
    
    return EWAFS

def Weighted_Average_Life_Test(clo_df,col):
    
    # This is CLO Specific!  It also is the Trigger, not the mapping
    # so this function was barking up the wrong tree.  I.e. you look up
    # the trigger in this map to compare with your test stat
    WAL_value = {'1/9/2021':9.00, '4/9/2021':8.67,'7/9/2021':8.42,'10/9/2021':8.17 ,
        '1/9/2022':7.92,'4/9/2022':7.67,'7/9/2022':7.42 ,'10/9/2022':7.17 ,'1/9/2023':6.92 ,
        '4/9/2023':6.67 ,'7/9/2023':6.42 ,'10/9/2023':6.17 ,'1/9/2024':5.92 ,'4/9/2024':5.67 ,
        '7/9/2024':5.42 ,'10/9/2024':5.17 ,'1/9/2025':4.92 ,'4/9/2025':4.67 ,'7/9/2025':4.42 ,
        '10/9/2025':4.17 ,'1/9/2026':3.92 ,'4/9/2026':3.67 ,'7/9/2026':3.42 ,'10/9/2026':3.17 ,
        '1/9/2027':2.92 ,'4/9/2027':2.67 ,'7/9/2027':2.42 ,'10/9/2027':2.17 ,'1/9/2028':1.92 ,
        '4/9/2028':1.67 ,'7/9/2028':1.42 ,'10/9/2028':1.17 ,'1/9/2029':0.92 ,'4/9/2029':0.67 ,
        '7/9/2029':0.42 ,'10/9/2029':0.17 ,'1/9/2030':0.00 ,'4/9/2030':0.00 ,'7/9/2030':0.00 ,
        '10/9/2030':0.00 ,'1/9/2031':0.00 ,'4/9/2031':0.00 ,'7/9/2031':0.00 ,'10/9/2031':0.00 ,
        '1/9/2032':0.00 ,'4/9/2032':0.00 ,'7/9/2032':0.00 ,'10/9/2032':0.00 ,'1/9/2033':0.00 ,
        '4/9/2033':0.00 ,'7/9/2033':0.00 ,'10/9/2033':0.0}
    
    
    
    WAL_val = pd.DataFrame.from_dict(WAL_value,orient='index',columns=['Value'])
    WAL_val.index = pd.to_datetime(WAL_val.index, dayfirst=False)
    clo_df['Maturity Date'] = pd.to_datetime(clo_df['Maturity Date'], infer_datetime_format=True)
    clo_df = pd.merge_asof(clo_df.sort_values('Maturity Date'), WAL_val, left_on='Maturity Date', 
                          right_on=WAL_val.index, direction='backward', suffixes=['', '_2'])

    clo_df['S&P CLO Specified Assets'] = clo_df[col].values
    mask = (clo_df['S&P Issuer Rating']=='CC') | \
           (clo_df['S&P Issuer Rating']=='SD') | \
           (clo_df['S&P Issuer Rating']=='NR') | \
           (clo_df['S&P Issuer Rating']=='D')
    clo_df.loc[mask,'S&P CLO Specified Assets'] = 0   # what about .isna()?

    WAL_stat = weighted_average(clo_df,cols=['S&P CLO Specified Assets','Value'])
    
    return WAL_stat


def percentage_CovLite(model_df,weight_col='Par_no_default'):  
    perC = model_df.loc[model_df['Cov Lite']=='Yes',[weight_col]].sum()/model_df[[weight_col]].sum()
    return perC.values[0]

def percentage_SecondLien(model_df,weight_col='Par_no_default'):  
    perC = model_df.loc[model_df['Lien Type']=='Second Lien',[weight_col]].sum()/model_df[[weight_col]].sum()
    return perC.values[0]

def percentage_C(model_df,weight_col='Par_no_default'):  
    perC = model_df.loc[model_df['Adjusted Moodys Rating'].str.match('C'),[weight_col]].sum()/model_df[[weight_col]].sum()
    return perC.values[0]

In [61]:
dict_2020_20 = {'Class A-1 Notes':240e6,'Class A-2 Notes':16e6,'Class B-1 Notes':39e6,'Class B-2 Notes':9e6,
               'Class C Notes':24e6,'Class D-1 Notes':16e6,'Class D-2 Notes':8e6,'Class E Notes':12e6}

def Overcollateralization_Stats(df,col,note_dict,format_output=True):
    PrinBal = df[col].sum()
    AB_Denom = note_dict['Class A-1 Notes']+note_dict['Class A-2 Notes']+note_dict['Class B-1 Notes']+\
                note_dict['Class B-2 Notes']
    C_Denom = AB_Denom + note_dict['Class C Notes']
    D_Denom = C_Denom + note_dict['Class D-1 Notes'] + note_dict['Class D-2 Notes']
    E_Denom = D_Denom + note_dict['Class E Notes']
    
    OC_stats_df = pd.DataFrame(np.nan,index=['Class A/B Overcollateralization Ratio',
        'Class C Overcollateralization Ratio',
        'Class D Overcollateralization Ratio',
        'Class E Overcollateralization Ratio',
        'Reinvestment Overcollateralization Ratio'],columns = [col])
    
    OC_stats_df.loc['Class A/B Overcollateralization Ratio',col] = PrinBal/AB_Denom
    OC_stats_df.loc['Class C Overcollateralization Ratio',col] = PrinBal/C_Denom
    OC_stats_df.loc['Class D Overcollateralization Ratio',col] = PrinBal/D_Denom
    OC_stats_df.loc['Class E Overcollateralization Ratio',col] = PrinBal/E_Denom
    OC_stats_df.loc['Reinvestment Overcollateralization Ratio',col] = PrinBal/E_Denom
    
    if format_output:
        OC_stats_df.loc['Class A/B Overcollateralization Ratio'] = \
            (OC_stats_df.loc['Class A/B Overcollateralization Ratio']*100).apply('{:.4f}%'.format)
        OC_stats_df.loc['Class C Overcollateralization Ratio'] = \
            (OC_stats_df.loc['Class C Overcollateralization Ratio']*100).apply('{:.4f}%'.format)
        OC_stats_df.loc['Class D Overcollateralization Ratio'] = \
            (OC_stats_df.loc['Class D Overcollateralization Ratio']*100).apply('{:.4f}%'.format)
        OC_stats_df.loc['Class E Overcollateralization Ratio'] = \
            (OC_stats_df.loc['Class E Overcollateralization Ratio']*100).apply('{:.4f}%'.format)
        OC_stats_df.loc['Reinvestment Overcollateralization Ratio'] = \
            (OC_stats_df.loc['Reinvestment Overcollateralization Ratio']*100).apply('{:.4f}%'.format)
   
    return OC_stats_df

# Read the Master Position Report

In [115]:
importlib.reload(ccu)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

In [116]:
path = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/CLO Reports/'
file = 'Master Position Report - BMS Aggregator as of 6.29.21.xlsx'
filepath = path + file

In [117]:
clo_df = ccu.get_master_position_report(filepath)

In [118]:
clo_df.columns

Index(['Parent Company', 'Issuer', 'Asset', 'CUSIP', 'ISIN', 'Asset Type',
       'Analyst', 'Spread', 'Floor', 'All In Rate', 'Maturity Date',
       'Mark Price', 'Adjusted Moodys Rating', 'WARF', 'MDPR', 'Moodys CFR',
       'Moodys Facility Rating', 'Moodys Issuer Outlook',
       'Moodys Issuer Watch', 'Moodys Recovery Rate', 'S&P Issuer Rating',
       'S&P Facility Rating', 'S&P Issuer Outlook', 'S&P Issuer Watch',
       'S&P Recovery', 'Agent Bank', 'CLO 4', 'CLO 5', 'CLO 6', 'CLO 7',
       'CLO 8R', 'CLO 9', 'CLO 10', 'CLO 11', 'CLO 12', 'CLO 13', 'CLO 14',
       'CLO 15', 'CLO 16', 'CLO 17', 'CLO 18', 'CLO 19', 'CLO 20', 'CLO 21',
       'Current Global Amount Outstanding', 'Moodys Industry', 'S&P Industry',
       'Lien Type', 'Issuer Country', 'Adj. WARF NEW',
       'S&P Recovery Rate (AAA)', 'S&P Global Ratings Factor', 'Libor Floor',
       'Effective Spread'],
      dtype='object')

In [119]:
clo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, LX190245 to LX185219
Data columns (total 54 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Parent Company                     478 non-null    object        
 1   Issuer                             478 non-null    object        
 2   Asset                              478 non-null    object        
 3   CUSIP                              453 non-null    object        
 4   ISIN                               438 non-null    object        
 5   Asset Type                         478 non-null    object        
 6   Analyst                            474 non-null    object        
 7   Spread                             472 non-null    float64       
 8   Floor                              466 non-null    float64       
 9   All In Rate                        478 non-null    float64       
 10  Maturity Date                  

In [185]:
clo_df['Lien Type'].unique()

array(['Second Lien', 'First Lien', nan], dtype=object)

# Collateral Quality Tests

In [84]:
import importlib
importlib.reload(ccu)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

In [125]:
pstats = ccu.Port_stats(clo_df, weight_col='CLO 20',format_output=False)
ocstats = ccu.Overcollateralization_Stats(clo_df,'CLO 20',ccu.dict_2020_20,format_output=False)
sstats = ccu.SP_CDO_Monitor_Test(clo_df,col='CLO 20')

pstats = pstats.append(ocstats)
pstats = pstats.append(sstats)
pstats

OP:  400000000.0 , NP:  397586678.61590004
Pass


,CLO 20
Min Floating Spread Test - no Libor Floors,3.669090e+00
Min Floating Spread Test - With Libor Floors,3.890123e+00
Minimum Weighted Average Coupon Test,4.005435e+00
Max Moodys Rating Factor Test (NEW WARF),2.778802e+03
Max Moodys Rating Factor Test (Orig WARF),2.929145e+03
Min Moodys Recovery Rate Test,4.773777e+01
Min S&P Recovery Rate Class A-1a,3.981928e+01
Moodys Diversity Test,8.106000e+01
Weighted Average Life Test,5.258967e+00
Percent Caa & lower,2.036717e+00


In [101]:
importlib.reload(ccu)
all_stats = ccu.all_stats_all_clos(clo_df,['CLO 4', 'CLO 5', 'CLO 6',
       'CLO 7', 'CLO 8R', 'CLO 9', 'CLO 10', 'CLO 11',
       'CLO 12', 'CLO 13', 'CLO 14', 'CLO 15',
       'CLO 16', 'CLO 17', 'CLO 18', 'CLO 19',
       'CLO 20', 'CLO 21'],ccu.dict_2020_20)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:422: RuntimeWarning: invalid value encountered in double_scalars
  wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols

OP:  600000000.0 , NP:  558490027.6645
Pass
OP:  400000000.0 , NP:  382196782.7675
Pass
OP:  1000000000.0 , NP:  937265481.7846
Pass
OP:  500000000.0 , NP:  492704968.9109
Pass


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:435: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['S&P CLO Specified Assets'] = clo_df[col].values
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Absolute Deviation'] = abs(clo_df['S&P Global Ratings Factor']-SPWARF)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

OP:  400000000.0 , NP:  354137689.7048
Pass
OP:  300000000.0 , NP:  308061426.0504
Pass
OP:  300000000.0 , NP:  272443395.9253
Pass
OP:  500000000.0 , NP:  493238985.80079997


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:422: RuntimeWarning: invalid value encountered in double_scalars
  wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols

Pass
OP:  500000000.0 , NP:  465143866.49329996
Pass
OP:  600000000.0 , NP:  583341833.0579
Pass
OP:  600000000.0 , NP:  592446875.0284
Pass
OP:  600000000.0 , NP:  585304468.1063


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:707: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Average Life']  = ((clo_df['Maturity Date'] - pd.Timestamp.today()).dt.days/365).round(2)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

Pass
OP:  500000000.0 , NP:  490312070.5497
Pass
OP:  500000000.0 , NP:  491682428.80579996
Pass
OP:  400000000.0 , NP:  396461784.9977
Pass
OP:  400000000.0 , NP:  392019385.4182
Pass
OP:  400000000.0 , NP:  389800706.96650004
Pass
OP:  500000000.0 , NP:  489059028.47440004
Pass


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:435: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['S&P CLO Specified Assets'] = clo_df[col].values
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Absolute Deviation'] = abs(clo_df['S&P Global Ratings Factor']-SPWARF)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [103]:
len(all_stats.index)

27

In [87]:
#percentage_C(clo_df,weight_col='CLO 2020-20')
clo_df.loc[(clo_df['Moodys CFR'].str.match('C'))&~clo_df['Moodys CFR'].isna(),['CLO 20']].sum()
clo_df.loc[(clo_df['MDPR'].str.match('C'))&~clo_df['MDPR'].isna(),['CLO 20']].sum()
clo_df.loc[(clo_df['Moodys Facility Rating'].str.match('C'))&~clo_df['Moodys Facility Rating'].isna(),['CLO 20']].sum()
clo_df.loc[(clo_df['Adjusted Moodys Rating'].str.match('C'))&~clo_df['Adjusted Moodys Rating'].isna(),['CLO 20']].sum()

#clo_df[['CLO 2020-20']].sum()

clo_df.loc[(clo_df['S&P Issuer Rating'].str.match('C'))&~clo_df['S&P Issuer Rating'].isna(),['CLO 20']].sum()/\
        clo_df[['CLO 20']].sum()


CLO 20    7.737976e+06
dtype: float64

CLO 20    9.670976e+06
dtype: float64

CLO 20    3776801.15
dtype: float64

CLO 20    2.748755e+07
dtype: float64

CLO 20    0.017613
dtype: float64

In [88]:
#clo_df[['Asset Type','CLO 2020-20','Spread']]
clo_df[['Asset Type','CLO 20','Spread']].info()
clo_df.loc[(clo_df['CLO 20'] > 0),['CLO 20']].info()
clo_df.loc[(clo_df['CLO 20'] == 0),['CLO 20']]

<class 'pandas.core.frame.DataFrame'>
Index: 474 entries, LX190245 to LX185219
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Asset Type  474 non-null    object 
 1   CLO 20      289 non-null    float64
 2   Spread      466 non-null    float64
dtypes: float64(2), object(1)
memory usage: 14.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 281 entries, LX193380 to LX185219
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CLO 20  281 non-null    float64
dtypes: float64(1)
memory usage: 4.4+ KB


,CLO 20
LXID,
LX192319,0.0
LX192678,0.0
LX167192,0.0
LX183733,0.0
LX179693,0.0
LX191880,0.0
LX155959,0.0
LX192014,0.0


In [89]:
# if no Asset Type then use LXID and str.match('US')
mask = (clo_df['Asset Type']!='Bond') & (clo_df['CLO 20'] > 0) & (~clo_df['Spread'].isna())
ccu.weighted_average(clo_df.loc[mask],['CLO 20','Spread'])
(clo_df.loc[mask,'CLO 20']*clo_df.loc[mask,'Spread']).sum()
clo_df.loc[mask,'CLO 20'].sum()


mask = (clo_df['Asset Type']=='Bond')
ccu.weighted_average(clo_df.loc[mask],['CLO 20','All In Rate'])
(clo_df.loc[mask,'CLO 20']*clo_df.loc[mask,'All In Rate']).sum()
clo_df.loc[mask,'CLO 20'].sum()


0.034290595137528156

13369402.02527225

389885389.0302

0.043111952366950895

293247.5

6802000.0

In [90]:
0.034290595-.001877

0.032413595

In [91]:
ccu.Excess_Weighted_Average_FloatSpread(clo_df,'CLO 20',0.032413595)

0.23886667534390452

In [92]:
#(np.floor((pd.to_datetime(df['dte']) - pd.to_datetime(dob)).dt.days / 365.25))


clo_df.loc[clo_df['Asset Type']=='Bond',['CLO 20']].sum() 
dict_2020_20 = {'Class A-1 Notes':240e6,'Class A-2 Notes':16e6,'Class B-1 Notes':39e6,'Class B-2 Notes':9e6,
               'Class C Notes':24e6,'Class D-1 Notes':16e6,'Class D-2 Notes':8e6,'Class E Notes':12e6}

CLO 20    6802000.0
dtype: float64

In [86]:
def prepost_Port_stats(model_df, cols):

    mask = abs(model_df[cols[0]]) > 0
    cstats = Port_stats(model_df.loc[mask], cols[0])
    cstats.rename(columns={'Portfolio Stats':cols[0]},inplace=True)

    if len(cols)>1:
        for c in cols[1:]:
            mask = abs(model_df[c]) > 0
            tstats = Port_stats(model_df.loc[mask], c)
            tstats.rename(columns={'Portfolio Stats':c},inplace=True)
            cstats = cstats.join(tstats)
    
    return cstats

In [22]:
ccu.prepost_Port_stats(clo_df, ['CLO 2013-4', 'CLO 2014-5', 'CLO 2014-6',
       'CLO 2014-7', 'CLO 2020-8R', 'CLO 2015-9', 'CLO 2015-10', 'CLO 2016-11',
       'CLO 2016-12', 'CLO 2017-13', 'CLO 2017-14', 'CLO 2018-15',
       'CLO 2019-16', 'CLO 2019-17', 'CLO 2020-18', 'CLO 2020-19',
       'CLO 2020-20', 'CLO 2021-21'])

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:685: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:695: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:695: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Average Life']  = ((clo_df['Maturity Date'] - pd.Timestamp.today()).dt.days/365).round(2)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:685: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:685: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:695: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

,CLO 2013-4,CLO 2014-5,CLO 2014-6,CLO 2014-7,CLO 2020-8R,CLO 2015-9,CLO 2015-10,CLO 2016-11,CLO 2016-12,CLO 2017-13,CLO 2017-14,CLO 2018-15,CLO 2019-16,CLO 2019-17,CLO 2020-18,CLO 2020-19,CLO 2020-20,CLO 2021-21
Min Floating Spread Test - no Libor Floors,3.43%,3.66%,3.61%,3.67%,3.62%,3.16%,3.17%,3.60%,3.36%,3.61%,3.61%,3.59%,3.61%,3.59%,3.57%,3.58%,3.66%,3.68%
Min Floating Spread Test - With Libor Floors,3.59%,3.84%,3.79%,3.85%,3.83%,3.38%,3.37%,3.78%,3.52%,3.79%,3.79%,3.78%,3.81%,3.77%,3.75%,3.75%,3.85%,3.88%
Max Moodys Rating Factor Test (NEW WARF),2853,2890,2848,2877,2830,2827,2826,2802,2809,2818,2828,2791,2815,2761,2757,2741,2762,2731
Max Moodys Rating Factor Test (Orig WARF),3052,3124,3083,3118,3094,3083,3045,3030,3006,3038,3047,3001,3030,2967,2968,2948,2948,2906
Min Moodys Recovery Rate Test,49.1%,48.1%,48.1%,48.1%,47.5%,48.9%,48.9%,48.1%,48.8%,48.1%,48.2%,48.2%,48.1%,48.2%,47.9%,47.8%,47.7%,48.1%
Min S&P Recovery Rate Class A-1a,42.7%,39.9%,39.9%,40.1%,39.3%,41.7%,42.0%,40.2%,41.2%,39.8%,39.8%,39.9%,40.0%,40.7%,40.6%,40.1%,40.0%,40.7%
Moodys Diversity Test,74,87,89,88,78,47,53,87,74,89,89,89,86,87,81,80,81,78
Weighted Average Life Test,3.94,4.89,4.86,4.79,4.86,2.99,3.03,4.84,4.07,4.85,4.90,4.93,4.97,5.06,5.15,5.14,5.27,5.36
Percent CCC & lower,13.1%,13.6%,13.2%,14.0%,12.7%,11.9%,12.9%,12.8%,13.0%,12.7%,11.7%,10.8%,11.1%,10.1%,9.7%,8.5%,6.9%,5.0%
Percent 2nd Lien,0.2%,0.7%,0.7%,0.8%,1.5%,0.1%,0.3%,0.7%,0.5%,0.9%,0.9%,0.9%,1.0%,0.8%,0.8%,1.0%,0.8%,0.7%


In [97]:
Weighted_Average_Life_Test(clo_df,col='CLO 2020-20')

3.315411722693518

In [110]:
# mask = 

clo_df.loc[(clo_df['S&P Issuer Rating']!='CC') &  (clo_df['S&P Issuer Rating']!='SD') &
           (clo_df['S&P Issuer Rating']!='NR') &  (clo_df['S&P Issuer Rating']!='D'),['CLO 2020-20','S&P Issuer Rating']]

,CLO 2020-20,S&P Issuer Rating
LXID,,
LX190245,NaN,B
LX190219,NaN,B
LX175827,NaN,B-
LX193380,6.000000e+05,B-
LX170018,1.220000e+05,B-
...,...,...
LX193419,3.810000e+05,B
LX185593,2.680188e+06,B
LX192354,1.908137e+06,B


In [84]:
clo_df = add_derived_features(clo_df)
clo_df[['Adj. WARF NEW','S&P Recovery Rate (AAA)','S&P Global Ratings Factor']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 474 entries, LX190245 to LX185219
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Adj. WARF NEW              467 non-null    float64
 1   S&P Recovery Rate (AAA)    471 non-null    float64
 2   S&P Global Ratings Factor  455 non-null    float64
dtypes: float64(3)
memory usage: 14.8+ KB


In [27]:
ccu.percentage_C(clo_df, 'CLO 2020-20')*100

6.929273429715044

In [38]:
def percentage_Caa(model_df,weight_col='CLO 20'):  
    
    perC = model_df.loc[model_df['MDPR'].str.match('C'),[weight_col]].sum()/model_df[[weight_col]].sum()
    return perC.values[0]

percentage_Caa(clo_df, 'CLO 20')*100

def percentage_CCC(model_df,weight_col='CLO 20'):  
    mask = (clo_df['S&P Issuer Rating']=='CC') |  (clo_df['S&P Issuer Rating']=='C') |  \
           (clo_df['S&P Issuer Rating']=='D') | (clo_df['S&P Issuer Rating']=='SD') | \
           (clo_df['S&P Issuer Rating']=='NR') #|  (clo_df['S&P Issuer Rating']=='D')
    #perC = model_df.loc[model_df['S&P Issuer Rating'].str.match('C'),[weight_col]].sum()/model_df[[weight_col]].sum()
    perC = model_df.loc[mask,[weight_col]].sum()/model_df[[weight_col]].sum()
    return perC.values[0]

percentage_CCC(clo_df, 'CLO 20')*100

2.4379337791259466

1.7360476420831505

## CDO Test stuff


In [192]:
def SP_CDO_Coefs(col='CLO 20'):
    """
    Looks up the coefficients in the dict table and returns the 3 as a float array
    """
    #clo_lists = ['CLO 18','CLO 16','CLO 19','CLO 17','CLO 20','CLO 8R','CLO 14',
    #             'CLO 13','CLO 15','CLO 10','CLO 11','CLO 12','CLO 7','CLO 6','CLO 5',
    #            'EUR 1', 'EUR 2', 'EUR 3', 'EUR 4']
    #     coefs_df = pd.DataFrame(np.nan,index=['C0','C1','C2'],columns = clo_list)
    
    clo_coefs = {'CLO 18': [0.121609,3.706211,1.025031],
                'CLO 16': [0.101745,4.769688,0.962975],
                'CLO 19': [0.097805,4.15832,1.071661],
                'CLO 17': [0.061931,4.266469,1.104848],
                'CLO 20': [0.109706,4.140929,1.064031],
                'CLO 8R': [0.106769,3.967347,0.997703],
                'CLO 14': [0.093957,4.215527,1.086073],
                'CLO 13': [0.066717,4.243131,1.11317],
                'CLO 15': [0.065108,4.620284,1.048429],
                'CLO 10': [-0.008128,3.480512,1.260735],
                'CLO 11': [0.059116,4.481791,1.10217],
                'CLO 12': [-0.046444,3.833393,1.107043],
                'CLO 7': [0.017043,4.541246,1.067143],
                'CLO 6': [0.05869,4.267761,1.099711],
                'CLO 5': [0.060439,4.275955,1.14419],
                'EUR 1': [0.185893082941552,3.70021989750327,0.881097473356921],
                'EUR 2': [0.206296268943602,4.05917015316102,0.947621203737026],
                'EUR 3': [0.191101158723369,3.39938112043418,0.952667210171681],
                'EUR 4': [np.nan,np.nan,np.nan]}
    
    return clo_coefs[col]

    
def SP_CDO_Type(col='CLO 20'):
    """
    There seems to be two types of BDR test stats
    Type II: 0.247621 + (SPWARF/9162.65) – (DRD/16757.2) – (ODM/7677.8) – (IDM/2177.56) – (RDM/34.0948) + (WAL/27.3896)
    Type I: 0.329915 + (1.210322 * EPDR) – (0.586627 * DRD) + (2.538684 /ODM) + (0.216729 / IDM) + (0.0575539 / RDM) – (0.0136662 * WAL)
    """
        
    clo_types = {'CLO 18': 'Type II',
                'CLO 16': 'Type II',
                'CLO 19': 'Type II',
                'CLO 17': 'Type II',
                'CLO 20': 'Type II',
                'CLO 8R': 'Type II',
                'CLO 14': 'Type I',
                'CLO 13': 'Type I',
                'CLO 15': 'Type I',
                'CLO 10': 'Type I',
                'CLO 11': 'Type I',
                'CLO 12': 'Type I',
                'CLO 7': 'Type I',
                'CLO 6': 'Type I',
                'CLO 5': 'Type I',
                'EUR 1': 'Type II',
                'EUR 2': 'Type I',
                'EUR 3': 'Type II',
                'EUR 4': 'Type II'}
    
    return clo_types[col]

In [209]:
def get_SPDR_Table(filepath):
    filepath = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/US CLO Triggers as of 6.24.21.xlsx'
    SPDR_Table = pd.read_excel(filepath,sheet_name = "SPDR")
    SPDR_Table.set_index('Tenor', inplace=True)
    return SPDR_Table.T

SPDR_Table = get_SPDR_Table(filepath)
SPDR_Table

Tenor,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
AAA,0.0,0.003249,0.015699,0.041484,0.084784,0.149746,0.240402,0.360599,0.513925,0.703660,...,6.467720,7.246658,8.066698,8.925853,9.821992,10.752863,11.716131,12.709401,13.730244,14.776220
AA+,0.0,0.008324,0.036996,0.091325,0.176281,0.296441,0.455938,0.658408,0.906953,1.204112,...,8.762054,9.692304,10.658664,11.658386,12.688687,13.746781,14.829898,15.935312,17.060358,18.202443
AA,0.0,0.017659,0.073622,0.172278,0.317753,0.513749,0.763415,1.069266,1.433135,1.856168,...,11.194685,12.255978,13.346459,14.462930,15.602275,16.761474,17.937621,19.127936,20.329775,21.540635
AA-,0.0,0.049443,0.139938,0.276841,0.464897,0.708173,1.009969,1.372767,1.798206,2.287090,...,12.530097,13.657463,14.810401,15.985473,17.179384,18.388990,19.611314,20.843553,22.083077,23.327436
A+,0.0,0.100435,0.257400,0.474538,0.755269,1.102407,1.517930,2.002861,2.557255,3.180245,...,14.902967,16.114039,17.342769,18.585784,19.839925,21.102252,22.370042,23.640779,24.912158,26.182066
A,0.0,0.198336,0.452472,0.770505,1.158808,1.621846,2.162163,2.780489,3.475934,4.246223,...,17.623250,18.927451,20.240163,21.558096,22.878270,24.197998,25.514868,26.826725,28.131652,29.427952
A-,0.0,0.305284,0.667329,1.100045,1.613532,2.213969,2.903924,3.682872,4.547804,5.493831,...,20.651699,22.041357,23.428880,24.811375,26.186325,27.551553,28.905184,30.245615,31.571487,32.881653
BBB+,0.0,0.403669,0.892889,1.484175,2.186032,3.000396,3.924151,4.950544,6.070420,7.273226,...,24.593206,26.062700,27.516624,28.952986,30.370173,31.766900,33.142161,34.495190,35.825422,37.132462
BBB,0.0,0.461619,1.091719,1.895696,2.867799,3.994693,5.258484,6.639097,8.116014,9.669463,...,29.502784,31.039941,32.545643,34.019346,35.460813,36.870044,38.247233,39.592717,40.906950,42.190470
BBB-,0.0,0.524294,1.445989,2.702054,4.229668,5.969443,7.867654,9.877442,11.959164,14.080160,...,37.184306,38.768990,40.301420,41.783417,43.216885,44.603759,45.945970,47.245417,48.503948,49.723352


In [265]:
def SPDR(df,SPDR_Table):
    
    df['Loan Life']  = ((df['Maturity Date'] - pd.Timestamp.today()).dt.days/365)
    
    badrates = pd.Series(['CC','NR'])
    
    def lookup_spdr(rating,loanlife):
        # this needs to skip any sub CCC- loans
        if rating!=rating:  # checking string is nan trick
            spdr = np.nan
        elif badrates.str.match(rating).any():
            spdr = np.nan
        else:
            tenors = (SPDR_Table.columns == loanlife//1) | (SPDR_Table.columns == loanlife//1 +1)
            spdr = SPDR_Table.loc[rating,tenors].values[0]+ loanlife%1 * \
                (SPDR_Table.loc[rating,tenors].values[1]-SPDR_Table.loc[rating,tenors].values[0])
        return df
        
    df['S&P Default Rating'] = df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
    
    return df
    
    

In [266]:
clo_df['Loan Life']  = ((clo_df['Maturity Date'] - pd.Timestamp.today()).dt.days/365)
#clo_df['Loan Life']

def lookup_spdr(rating,loanlife):
    
    print(rating)
    if rating!=rating:  # checking string is nan trick
        spdr = np.nan
    elif badrates.str.match(rating).any():
        spdr = np.nan
    else:
        tenors = (SPDR_Table.columns == loanlife//1) | (SPDR_Table.columns == loanlife//1 +1)
        spdr = SPDR_Table.loc[rating,tenors].values[0]+ loanlife%1 * \
            (SPDR_Table.loc[rating,tenors].values[1]-SPDR_Table.loc[rating,tenors].values[0])


    return spdr

clo_df[['S&P Facility Rating','Loan Life']].info()

clo_df[['S&P Facility Rating','Loan Life']].apply(lambda x: lookup_spdr(x[0],x[1]),axis=1)
#SPDR(clo_df,SPDR_Table)

#clo_df['S&P Default Rating']

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, LX190245 to LX185219
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   S&P Facility Rating  404 non-null    object 
 1   Loan Life            478 non-null    float64
dtypes: float64(1), object(1)
memory usage: 27.4+ KB
CCC+
B
B-
nan
B-
B-
B
B
nan
B
B
B+
B
B
B
B
B
B+
nan
nan
B+
nan
nan
BB
BB
BB
B+
B-
B-
B-
B-
BB
B
B
B
B-
BB+
BB+
nan
nan
nan
B-
B-
BBB-
BB-
B+
B+
B+
B+
B
B
BB+
BB
nan
B-
nan
B
B+
nan
nan
nan
CC
CCC
BB
BB
BB-
B
BB+
nan
BB+
B-
BBB-
BB+
B
CCC+
B-
B-
B-
B-
BB
B-
B-
B
B
BB-
B
B
BBB-
B+
B-
BB+
BB+
BB+
BB+
B
B
CCC+
B
B
B-
B
nan
B
B
B-
B+
BBB-
BBB-
B
B
B
nan
B+
B+
nan
nan
nan
B-
B+
CCC
B
B
B
B-
B+
CCC+
B
B
B
CCC-
B-
CCC
B
B
nan
B
B
B+
B
B-
B-
B
B
B
nan
B+
CCC+
B-
BB+
BB+
BBB-
BBB-
B-
CCC+
B
nan
B
nan
nan
B
CCC+
CCC+
CCC-
BB-
BB-
B
CCC
B-
B
BB-
nan
nan
BB+
B
nan
B
CCC+
B-
B-
B
nan
B-
B-
B
B
B
B
nan
nan
nan
BB
nan
B+
nan
nan
B+
B+
B-
nan
BB

LXID
LX190245    59.143323
LX190219    36.887557
LX175827    35.238250
LX193380          NaN
LX170018    30.760726
              ...    
LX193419    28.977813
LX185593    34.165186
LX192354    32.274433
LX167827    28.276876
LX185219    29.304118
Length: 478, dtype: float64

In [224]:
SPDR_Table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, AAA to CCC-
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       19 non-null     float64
 1   1       19 non-null     float64
 2   2       19 non-null     float64
 3   3       19 non-null     float64
 4   4       19 non-null     float64
 5   5       19 non-null     float64
 6   6       19 non-null     float64
 7   7       19 non-null     float64
 8   8       19 non-null     float64
 9   9       19 non-null     float64
 10  10      19 non-null     float64
 11  11      19 non-null     float64
 12  12      19 non-null     float64
 13  13      19 non-null     float64
 14  14      19 non-null     float64
 15  15      19 non-null     float64
 16  16      19 non-null     float64
 17  17      19 non-null     float64
 18  18      19 non-null     float64
 19  19      19 non-null     float64
 20  20      19 non-null     float64
 21  21      19 non-null     float64
 22  22   

In [204]:
LL = ((clo_df['Maturity Date'] - pd.Timestamp.today()).dt.days/365)
LL
LL//1
LL%1

LXID
LX190245    7.342466
LX190219    6.345205
LX175827    4.241096
LX193380    6.794521
LX170018    3.495890
              ...   
LX193419    6.739726
LX185593    5.676712
LX192354    5.238356
LX167827    3.104110
LX185219    6.821918
Name: Maturity Date, Length: 478, dtype: float64

LXID
LX190245    7.0
LX190219    6.0
LX175827    4.0
LX193380    6.0
LX170018    3.0
           ... 
LX193419    6.0
LX185593    5.0
LX192354    5.0
LX167827    3.0
LX185219    6.0
Name: Maturity Date, Length: 478, dtype: float64

LXID
LX190245    0.342466
LX190219    0.345205
LX175827    0.241096
LX193380    0.794521
LX170018    0.495890
              ...   
LX193419    0.739726
LX185593    0.676712
LX192354    0.238356
LX167827    0.104110
LX185219    0.821918
Name: Maturity Date, Length: 478, dtype: float64

In [218]:
SPDR_Table.columns
mask = (SPDR_Table.columns == LL[0]//1) | (SPDR_Table.columns == LL[0]//1 +1)
SPDR_Table.loc['B+',mask].values[0]

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64', name='Tenor')

30.011114

In [193]:
SP_CDO_Coefs('CLO 6')[1]
SP_CDO_Type('CLO 20')

4.267761

'Type II'

In [260]:
sstat = SP_CDO_Monitor_Test(clo_df,col='CLO 2020-20')

OP:  400000000.0 , NP:  389800706.96650004
Pass


,CLO 2020-20
S&P Weighted Average Rating Factor (SP WARF),2735.926570
Default Rate Dispersion (DRD),647.482947
Obligor Diversity Measure (ODM),185.502110
Industry Diversity Measure (IDM),16.176295
Regional Diversity Measure (RDM),1.339561
Weighted Average Life (WAL),5.280083
Break-Even Default Rate (BDR),0.674757
Adjusted Break-Even Default Rate (Adj BDR),0.736013
Scenario Default Rate (SDR),0.629476


In [115]:
clo_df[]

<class 'pandas.core.frame.DataFrame'>
Index: 474 entries, LX190245 to LX185219
Data columns (total 53 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Parent Company                     474 non-null    object        
 1   Issuer                             474 non-null    object        
 2   Asset                              474 non-null    object        
 3   CUSIP                              447 non-null    object        
 4   ISIN                               436 non-null    object        
 5   Asset Type                         474 non-null    object        
 6   Analyst                            466 non-null    object        
 7   Spread                             466 non-null    float64       
 8   Floating Spread Floor              459 non-null    float64       
 9   All In Rate                        474 non-null    float64       
 10  Maturity Date                  

# CLO Triggers

In [61]:
pathT = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/'
fileT = 'US CLO Triggers as of 6.24.21.xlsx'
filepathT = pathT + fileT

In [62]:
triggers = pd.read_excel(filepathT)
triggers

,DatasetID,ReportKey,Counter,ScenarioKey,PassFailStatus,Requirement,Outcome,Title,MarketValue,TotalCap,...,CLOName,CLORank,TestName,TestResult,TestTrigger,Variance,Include,TestType,Limitation,TestRank
0,328,CMPTstGeneric50.clsTstGeneric_09_01_01,1,1,In Compliance,>= 115.00%,2.4751,Interest Coverage Test - Class B,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class B,2.4751,>= 115.00%,1.1615,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
1,328,CMPTstGeneric50.clsTstGeneric_09_01_01_01,1,1,In Compliance,>= 110.00%,2.1547,Interest Coverage Test - Class C,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class C,2.1547,>= 110.00%,1.1110,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
2,328,CMPTstGeneric50.clsTstGeneric_09_01_01_01_01,1,1,In Compliance,>= 105.00%,1.8116,Interest Coverage Test - Class D,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class D,1.8116,>= 105.00%,1.0605,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
3,328,CMPTstGeneric50.clsTstGeneric_09_01_01_02,1,1,In Compliance,>= 120.00%,2.8343,Interest Coverage Test - Class A,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class A,2.8343,>= 120.00%,1.2120,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
4,328,CMPTstGeneric7.clsTstGeneric,1,1,In Compliance,<= 5.0%,0.0000,(13) Moody's Counterparty Criteria - Aggregate...,NaN,NaN,...,CLO 6,3,(13) Moody's Counterparty Criteria - Aggregate...,0.0000,<= 5.0%,0.0495,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,3,1,In Compliance,<= 1.0%,0.0020,(03)(c) Single Obligor - Senior Secured Bonds ...,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0020,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1880,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,4,1,In Compliance,<= 1.0%,0.0020,(03)(c) Single Obligor - Senior Secured Bonds ...,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0020,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1881,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,5,1,In Compliance,<= 1.0%,0.0010,(03)(c) Single Obligor - Senior Secured Bonds ...,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0010,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1882,1281,CMPTstGeneric7.clsTstSells,1,1,In Compliance,<= 30.0%,0.1810,Discretionary Sales,NaN,NaN,...,CLO 18,15,Discretionary Sales,0.1810,<= 30.0%,0.2970,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3


In [63]:
triggers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1884 entries, 0 to 1883
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DatasetID         1884 non-null   int64         
 1   ReportKey         1884 non-null   object        
 2   Counter           1884 non-null   int64         
 3   ScenarioKey       1884 non-null   int64         
 4   PassFailStatus    1883 non-null   object        
 5   Requirement       1884 non-null   object        
 6   Outcome           1884 non-null   float64       
 7   Title             1884 non-null   object        
 8   MarketValue       0 non-null      float64       
 9   TotalCap          0 non-null      float64       
 10  ScenarioName      1884 non-null   object        
 11  IsRefreshed       1884 non-null   bool          
 12  RequirementRaw    1884 non-null   float64       
 13  OutcomeRaw        1884 non-null   float64       
 14  RecordSource      1884 n

In [64]:
clo_list = triggers['CLOName'].unique()
#triggers[['CLOName','Title','RequirementRaw','OutcomeRaw']].groupby(['CLOName','Title']).sum()
#clo_list = clo_list.sort()
clo_list = sorted(clo_list)

clo_list = clo_list[11:]+clo_list[0:10]
clo_list

['CLO 4',
 'CLO 5',
 'CLO 6',
 'CLO 7',
 'CLO 8R',
 'CLO 9',
 'CLO 10',
 'CLO 11',
 'CLO 12',
 'CLO 13',
 'CLO 14',
 'CLO 15',
 'CLO 16',
 'CLO 17',
 'CLO 18',
 'CLO 19']

In [65]:
len(triggers['Title'].unique())
triggers['Title'].count()

767

1884

In [66]:
WAL_Trigger_Table = triggers.loc[triggers['Title']=='Weighted Average Life',['CLOName','PassFailStatus','Requirement','Outcome']]

WAL_Trigger_Table

,CLOName,PassFailStatus,Requirement,Outcome
108,CLO 6,In Compliance,<= 5.32,4.77
218,CLO 7,In Compliance,<= 4.70,4.70
325,CLO 10,Out of Compliance,<= 2.33,2.98
427,CLO 9,Out of Compliance,<= 2.02,2.93
539,CLO 11,In Compliance,<= 5.13,4.76
660,CLO 13,In Compliance,<= 5.07,4.76
781,CLO 14,In Compliance,<= 5.49,4.82
887,CLO 5,In Compliance,<= 5.84,4.80
1008,CLO 15,In Compliance,<= 6.00,4.84
1136,CLO 4,Out of Compliance,<= 3.84,3.87


In [67]:
# Quality Tests
#Minimum Weighted Average S&P Recovery Rate Test - Class A-1
#Minimum Floating Spread Test
#Minimum Weighted Average Coupon Test
#Moody's Diversity Test
#Maximum Moody's Rating Factor Test
#Minimum Weighted Average Moody’s Recovery Rate Test
#Weighted Average Life
cols = ['CLOName','PassFailStatus','Requirement','Outcome']

WAL_Trigger_Table = triggers.loc[triggers['Title']=='Weighted Average Life',cols]
WAS_Trigger_Table = triggers.loc[triggers['Title']=='Minimum Floating Spread Test',cols]
SPWARR_Trigger_Table = triggers.loc[triggers['Title']=='Minimum Weighted Average S&P Recovery Rate Test - Class A-1',cols]
WAC_Trigger_Table = triggers.loc[triggers['Title']=='Minimum Weighted Average Coupon Test',cols]
DIV_Trigger_Table = triggers.loc[triggers['Title']=='Moody\'s Diversity Test',cols]
WARF_Trigger_Table = triggers.loc[triggers['Title']=='Maximum Moody\'s Rating Factor Test',cols]
MWARR_Trigger_Table = triggers.loc[triggers['Title']=='Minimum Weighted Average Moody’s Recovery Rate Test',cols]

In [68]:
WAS_Trigger_Table
SPWARR_Trigger_Table
WAC_Trigger_Table
DIV_Trigger_Table
WARF_Trigger_Table
MWARR_Trigger_Table

,CLOName,PassFailStatus,Requirement,Outcome
13,CLO 6,In Compliance,0.031,0.036498
121,CLO 7,In Compliance,0.0345,0.037066
232,CLO 10,In Compliance,0.03,0.032301
339,CLO 9,Out of Compliance,0.0325,0.032341
440,CLO 11,In Compliance,0.035,0.036431
552,CLO 13,In Compliance,0.035,0.036549
672,CLO 14,In Compliance,0.0345,0.036700
795,CLO 5,In Compliance,0.032,0.037009
899,CLO 15,In Compliance,0.0345,0.036424
1024,CLO 4,In Compliance,0.0325,0.034408


,CLOName,PassFailStatus,Requirement,Outcome
6,CLO 6,In Compliance,>= 38.0%,0.3810
114,CLO 7,In Compliance,>= 40.2%,0.4030
225,CLO 10,In Compliance,>= 40.00%,0.4009
332,CLO 9,Out of Compliance,>= 40.00%,0.3985
433,CLO 11,In Compliance,>= 38.00%,0.3835
545,CLO 13,In Compliance,>= 38.0%,0.3800
666,CLO 14,In Compliance,>= 38.00%,0.3810
788,CLO 5,Out of Compliance,>= 38.00%,0.3799
893,CLO 15,In Compliance,>= 40.00%,0.4017
1143,CLO 12,In Compliance,>= 41.00%,0.4150


,CLOName,PassFailStatus,Requirement,Outcome
16,CLO 6,In Compliance,0.075,0.000000
124,CLO 7,In Compliance,0.065,0.000000
235,CLO 10,In Compliance,0.065,0.000000
342,CLO 9,In Compliance,0.065,0.000000
443,CLO 11,In Compliance,0.065,0.000000
555,CLO 13,In Compliance,0.065,0.000000
675,CLO 14,In Compliance,0.065,0.000000
798,CLO 5,In Compliance,0.065,0.000000
902,CLO 15,In Compliance,0.065,0.000000
1030,CLO 4,In Compliance,0.075,0.000000


,CLOName,PassFailStatus,Requirement,Outcome
18,CLO 6,In Compliance,>= 60,86.0
126,CLO 7,In Compliance,>= 80,86.0
237,CLO 10,Out of Compliance,>= 65,52.0
344,CLO 9,Out of Compliance,>= 60,46.0
445,CLO 11,In Compliance,>= 80,85.0
557,CLO 13,NaN,>= 80,87.0
677,CLO 14,In Compliance,>= 80,86.0
800,CLO 5,In Compliance,>= 80,85.0
904,CLO 15,In Compliance,>= 80,86.0
1033,CLO 4,Out of Compliance,>= 75,72.0


,CLOName,PassFailStatus,Requirement,Outcome
21,CLO 6,In Compliance,<= 3283,3074.0
129,CLO 7,In Compliance,<= 3236,3107.0
240,CLO 10,In Compliance,<= 3200,3052.0
347,CLO 9,Out of Compliance,<= 2883,3094.0
448,CLO 11,In Compliance,<= 3037,3025.0
560,CLO 13,In Compliance,<= 3050,3032.0
680,CLO 14,In Compliance,<= 3158,3044.0
803,CLO 5,In Compliance,<= 3377,3116.0
907,CLO 15,In Compliance,<= 3063,3001.0
1038,CLO 4,In Compliance,<= 3387,3068.0


,CLOName,PassFailStatus,Requirement,Outcome
23,CLO 6,In Compliance,>= 43.0%,0.4810
131,CLO 7,In Compliance,>= 43.00%,0.4807
243,CLO 10,In Compliance,>= 43.0%,0.4900
349,CLO 9,In Compliance,>= 46.50%,0.4885
451,CLO 11,In Compliance,>= 43.00%,0.4803
563,CLO 13,In Compliance,>= 43.00%,0.4801
683,CLO 14,In Compliance,>= 43.00%,0.4813
805,CLO 5,In Compliance,>= 43.0%,0.4820
910,CLO 15,In Compliance,>= 43.00%,0.4812
1040,CLO 4,In Compliance,>= 43.0%,0.4910


In [69]:
s = 'Minimum Weighted Average S&P Recovery Rate Test - Class A-1'
clo = 'CLO 4'

#triggers.loc[(triggers['Title']==s)&(triggers['CLOName']==clo),'Requirement'].values[0]
triggers.loc[(triggers['CLOName']==clo),['Title','Requirement']].values[0]

array(['Class B Interest Coverage Test', '>= 105.00%'], dtype=object)

In [96]:

qstats_list = ['Minimum Weighted Average S&P Recovery Rate Test - Class A-1',
    'Minimum Floating Spread Test',
    'Minimum Weighted Average Coupon Test',
    'Moody\'s Diversity Test',
    'Maximum Moody\'s Rating Factor Test',
    'Minimum Weighted Average Moody’s Recovery Rate Test',
    'Weighted Average Life']
#clo_list

def build_trigger_tables(trigger_df,clo_list,stats_list):
    trigger_table = pd.DataFrame(np.nan,index=stats_list,columns = clo_list)
    for clo in clo_list:
        for s in stats_list:
            #print('s: ',s,' clo: ',clo)
            try:
                trigger_table.loc[s,clo] = trigger_df.loc[(trigger_df['Title']==s)&
                                                      (trigger_df['CLOName']==clo),'RequirementRaw'].values[0]
            except:
                print(s,' for ',clo, ' DNE')
                #trigger_table.loc[s,clo] = np.nan
    
    return trigger_table

#trigger_table = 
trigger_table_q = build_trigger_tables(triggers,clo_list,qstats_list)
trigger_table_q

Minimum Weighted Average S&P Recovery Rate Test - Class A-1  for  CLO 4  DNE
Minimum Weighted Average Moody’s Recovery Rate Test  for  CLO 16  DNE


,CLO 4,CLO 5,CLO 6,CLO 7,CLO 8R,CLO 9,CLO 10,CLO 11,CLO 12,CLO 13,CLO 14,CLO 15,CLO 16,CLO 17,CLO 18,CLO 19
Minimum Weighted Average S&P Recovery Rate Test - Class A-1,NaN,0.380,0.380,0.4020,0.40,0.4000,0.400,0.380,0.410,0.380,0.3800,0.4000,0.400,0.4100,0.405,0.40
Minimum Floating Spread Test,0.0325,0.032,0.031,0.0345,0.02,0.0325,0.030,0.035,0.032,0.035,0.0345,0.0345,0.020,0.0345,0.020,0.02
Minimum Weighted Average Coupon Test,0.0750,0.065,0.075,0.0650,0.07,0.0650,0.065,0.065,0.065,0.065,0.0650,0.0650,0.065,0.0700,0.070,0.07
Moody's Diversity Test,75.0000,80.000,60.000,80.0000,30.00,60.0000,65.000,80.000,70.000,80.000,80.0000,80.0000,40.000,75.0000,40.000,50.00
Maximum Moody's Rating Factor Test,3387.0000,3377.000,3283.000,3236.0000,3400.00,2883.0000,3200.000,3037.000,3400.000,3050.000,3158.0000,3063.0000,3350.000,2999.0000,3400.000,3400.00
Minimum Weighted Average Moody’s Recovery Rate Test,0.4300,0.430,0.430,0.4300,0.43,0.4650,0.430,0.430,0.430,0.430,0.4300,0.4300,NaN,0.4300,0.430,0.43
Weighted Average Life,3.8400,5.840,5.320,4.7000,8.00,2.0200,2.330,5.130,3.810,5.070,5.4900,6.0000,7.000,7.2500,7.250,6.68


In [146]:
#triggers.loc[triggers['TestType']=='Coverage Test',['CLOName','PassFailStatus','Requirement','Outcome']]
(triggers.loc[triggers['TestType']=='Coverage Test','Title'].unique())

array(['Interest Coverage Test - Class B',
       'Interest Coverage Test - Class C',
       'Interest Coverage Test - Class D',
       'Interest Coverage Test - Class A',
       'Overcollateralization Ratio Test - Class B',
       'Overcollateralization Ratio Test - Class C',
       'Overcollateralization Ratio Test - Class D',
       'Overcollateralization Ratio Test - Class A',
       'Reinvestment Overcollateralization Test', nan,
       'Overcollateralization Ratio Test - Event of Default',
       'Interest Diversion Test',
       'Overcollateralization Ratio Test - Class E',
       'Interest Coverage Test - Class A/B',
       'Overcollateralization Ratio Test - Class A/B'], dtype=object)

In [99]:
coverage_stats = ['Interest Coverage Test - Class A',
       'Interest Coverage Test - Class B',
       'Interest Coverage Test - Class A/B',
       'Interest Coverage Test - Class C',
       'Interest Coverage Test - Class D',
       'Interest Diversion Test', 
       'Overcollateralization Ratio Test - Class A',
       'Overcollateralization Ratio Test - Class B',
       'Overcollateralization Ratio Test - Class A/B',
       'Overcollateralization Ratio Test - Class C',
       'Overcollateralization Ratio Test - Class D',
       'Overcollateralization Ratio Test - Class E',
       'Reinvestment Overcollateralization Test',
       'Overcollateralization Ratio Test - Event of Default']
       
coverage_map = {'Interest Coverage Test - Class A':'Interest Coverage Test - Class A',
       'Interest Coverage Test - Class B':'Interest Coverage Test - Class B',
       'Interest Coverage Test - Class A/B':'Interest Coverage Test - Class A/B',
       'Interest Coverage Test - Class C':'Interest Coverage Test - Class C',
       'Interest Coverage Test - Class D':'Interest Coverage Test - Class D',
       'Interest Diversion Test':'Interest Diversion Test', 
       'Overcollateralization Ratio Test - Class A':'Overcollateralization Ratio Test - Class A',
       'Overcollateralization Ratio Test - Class B':'Overcollateralization Ratio Test - Class B',
       'Overcollateralization Ratio Test - Class A/B':'Overcollateralization Ratio Test - Class A/B',
       'Overcollateralization Ratio Test - Class C':'Overcollateralization Ratio Test - Class C',
       'Overcollateralization Ratio Test - Class D':'Overcollateralization Ratio Test - Class D',
       'Overcollateralization Ratio Test - Class E':'Overcollateralization Ratio Test - Class E',
       'Reinvestment Overcollateralization Test':'Reinvestment Overcollateralization Test',
       'Overcollateralization Ratio Test - Class A1 [Event of Default - Section 5.1(g)':
                'Overcollateralization Ratio Test - Event of Default',
       'Overcollateralization Ratio Test - Event of Default':'Overcollateralization Ratio Test - Event of Default',
       'Class B Interest Coverage Test':'Interest Coverage Test - Class B',  #
       'Class C Interest Coverage Test': 'Interest Coverage Test - Class C',
       'Class D Interest Coverage Test':'Interest Coverage Test - Class D',
       'Class A Interest Coverage Test':'Interest Coverage Test - Class A',
       'Class B Overcollateralization Ratio Test':'Overcollateralization Ratio Test - Class B',
       'Class C Overcollateralization Ratio Test':'Overcollateralization Ratio Test - Class C',
       'Class D Overcollateralization Ratio Test':'Overcollateralization Ratio Test - Class D',
       'Class A Overcollateralization Ratio Test':'Overcollateralization Ratio Test - Class A',
       'Class E Overcollateralization Ratio Test':'Overcollateralization Ratio Test - Class E',
       'Class A-1 Overcollateralization Ratio Test [Event of Default - Section 5.1(g)]':
               'Overcollateralization Ratio Test - Event of Default'}
       
       
triggers.loc[triggers['TestType']=='Coverage Test','Title'] = triggers.loc[triggers['TestType']=='Coverage Test','Title'].map(coverage_map)


In [157]:
#Moody's Default Probability Rating Of Caa1 or Below
con_stats = ['Cov-Lite Loans','Moody\'s Default Probability Rating Of Caa1 or Below',
    'Moody\'s Default Probability Rating Of Caa1 or Below and/or S&P Rating of CCC+ or Below',
    'Moody\'s Rating <= Caa1','S&P Rating <= CCC+',
    'S&P Rating of CCC+ or Below']

con_stats_map = {'Cov-Lite Loans':'Cov-Lite Loans','Moody\'s Default Probability Rating Of Caa1 or Below':
                 'Moody\'s Rating <= Caa1','Moody\'s Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+':
                 'Moody\'s Default Probability Rating Of Caa1 or Below and/or S&P Rating of CCC+ or Below',
                 'Moody\'s Rating <= Caa1':'Moody\'s Rating <= Caa1','S&P Rating <= CCC+',
                 'S&P Rating of CCC+ or Below':'S&P Rating <= CCC+'}

#con_stats = [x.lstrip("(0123456789)/ ") for x in con_stats]

triggers.loc[triggers['TestType']=='Coverage Test','Title'] = triggers.loc[triggers['TestType']=='Coverage Test','Title'].map(coverage_map)

In [165]:
#[x.lstrip("(0123456789)/ ") for x in con_stats]
importlib.reload(ccu)
triggers = ccu.get_triggers(filepathT)
triggers

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

,DatasetID,ReportKey,Counter,ScenarioKey,PassFailStatus,Requirement,Outcome,Title,MarketValue,TotalCap,...,CLOName,CLORank,TestName,TestResult,TestTrigger,Variance,Include,TestType,Limitation,TestRank
0,328,CMPTstGeneric50.clsTstGeneric_09_01_01,1,1,In Compliance,>= 115.00%,2.4751,Interest Coverage Test - Class B,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class B,2.4751,>= 115.00%,1.1615,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
1,328,CMPTstGeneric50.clsTstGeneric_09_01_01_01,1,1,In Compliance,>= 110.00%,2.1547,Interest Coverage Test - Class C,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class C,2.1547,>= 110.00%,1.1110,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
2,328,CMPTstGeneric50.clsTstGeneric_09_01_01_01_01,1,1,In Compliance,>= 105.00%,1.8116,Interest Coverage Test - Class D,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class D,1.8116,>= 105.00%,1.0605,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
3,328,CMPTstGeneric50.clsTstGeneric_09_01_01_02,1,1,In Compliance,>= 120.00%,2.8343,Interest Coverage Test - Class A,NaN,NaN,...,CLO 6,3,Interest Coverage Test - Class A,2.8343,>= 120.00%,1.2120,NaN,Coverage Test,Interst Proceeds are used to make payments in ...,1
4,328,CMPTstGeneric7.clsTstGeneric,1,1,In Compliance,<= 5.0%,0.0000,NaN,NaN,NaN,...,CLO 6,3,(13) Moody's Counterparty Criteria - Aggregate...,0.0000,<= 5.0%,0.0495,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,3,1,In Compliance,<= 1.0%,0.0020,NaN,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0020,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1880,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,4,1,In Compliance,<= 1.0%,0.0020,NaN,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0020,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1881,1281,CMPTstGeneric7.clsTstGeneric_CL_35_02_01,5,1,In Compliance,<= 1.0%,0.0010,NaN,NaN,NaN,...,CLO 18,15,(03)(c) Single Obligor - Senior Secured Bonds ...,0.0010,<= 1.0%,0.0099,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3
1882,1281,CMPTstGeneric7.clsTstSells,1,1,In Compliance,<= 30.0%,0.1810,NaN,NaN,NaN,...,CLO 18,15,Discretionary Sales,0.1810,<= 30.0%,0.2970,NaN,Concentration Limitation Test,Test must be maintained or improved after givi...,3


In [163]:
con_stats = ['Cov-Lite Loans',
    'Moody\'s Default Probability Rating Of Caa1 or Below and/or S&P Rating of CCC+ or Below',
    'Moody\'s Rating <= Caa1','S&P Rating <= CCC+']

In [167]:
trigger_table_con = build_trigger_tables(triggers,clo_list,ccu.con_stats)
trigger_table_con

Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 5  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 6  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 7  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 8R  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 9  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 10  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 11  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 12  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 13  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 14  DNE
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+  for  CLO 15  DNE
Moody's Default Probability Rating <= Caa1 and/

,CLO 4,CLO 5,CLO 6,CLO 7,CLO 8R,CLO 9,CLO 10,CLO 11,CLO 12,CLO 13,CLO 14,CLO 15,CLO 16,CLO 17,CLO 18,CLO 19,CLO 20
Cov-Lite Loans,0.500,0.600,0.600,0.700,0.650,0.500,0.500,0.600,0.500,0.500,0.600,0.600,0.650,0.600,0.600,0.600,0.600
Moody's Default Probability Rating <= Caa1 and/or S&P Rating <= CCC+,0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Moody's Rating <= Caa1,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075
S&P Rating <= CCC+,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075,0.075


In [94]:
all_stats
trigger_table

,CLO 4,CLO 5,CLO 6,CLO 7,CLO 8R,CLO 9,CLO 10,CLO 11,CLO 12,CLO 13,CLO 14,CLO 15,CLO 16,CLO 17,CLO 18,CLO 19,CLO 20,CLO 21
Min Floating Spread Test - no Libor Floors,3.43%,3.66%,3.61%,3.67%,3.62%,3.16%,3.17%,3.60%,3.36%,3.61%,3.61%,3.59%,3.61%,3.59%,3.57%,3.58%,3.66%,3.68%
Min Floating Spread Test - With Libor Floors,3.59%,3.84%,3.79%,3.85%,3.83%,3.38%,3.37%,3.78%,3.52%,3.79%,3.79%,3.78%,3.81%,3.77%,3.75%,3.75%,3.85%,3.88%
Minimum Weighted Average Coupon Test,nan%,nan%,nan%,nan%,4.20%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,4.57%,nan%,4.30%,nan%,4.31%,4.12%
Max Moodys Rating Factor Test (NEW WARF),2853,2890,2848,2877,2830,2827,2826,2802,2809,2818,2828,2791,2815,2761,2757,2741,2762,2731
Max Moodys Rating Factor Test (Orig WARF),3052,3124,3083,3118,3094,3083,3045,3030,3006,3038,3047,3001,3030,2967,2968,2948,2948,2906
Min Moodys Recovery Rate Test,49.1%,48.1%,48.1%,48.1%,47.5%,48.9%,48.9%,48.1%,48.8%,48.1%,48.2%,48.2%,48.1%,48.2%,47.9%,47.8%,47.7%,48.1%
Min S&P Recovery Rate Class A-1a,42.7%,39.9%,39.9%,40.1%,39.3%,41.7%,42.0%,40.2%,41.2%,39.8%,39.8%,39.9%,40.0%,40.7%,40.6%,40.1%,40.0%,40.7%
Moodys Diversity Test,74,87,89,88,78,47,53,87,74,89,89,89,86,87,81,80,81,78
Weighted Average Life Test,3.94,4.89,4.86,4.79,4.86,2.98,3.03,4.84,4.07,4.85,4.90,4.93,4.97,5.06,5.15,5.14,5.26,5.36
Percent Caa & lower,6.8%,7.6%,7.2%,7.2%,5.6%,6.7%,8.5%,6.8%,6.9%,6.7%,6.1%,5.0%,5.1%,4.4%,3.6%,2.8%,2.4%,1.7%


,CLO 4,CLO 5,CLO 6,CLO 7,CLO 8R,CLO 9,CLO 10,CLO 11,CLO 12,CLO 13,CLO 14,CLO 15,CLO 16,CLO 17,CLO 18,CLO 19
Interest Coverage Test - Class A,1.1000,1.200,1.2000,1.200,1.200,1.200,1.2000,1.200,1.2000,1.2000,1.200,1.2000,NaN,NaN,NaN,NaN
Interest Coverage Test - Class B,1.0500,1.150,1.1500,1.150,1.100,1.150,1.1500,1.150,1.1500,1.1500,1.150,1.1500,NaN,NaN,NaN,NaN
Interest Coverage Test - Class A/B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2000,1.2000,1.2000,1.2000
Interest Coverage Test - Class C,1.0200,1.100,1.1000,1.100,1.050,1.100,1.1000,1.100,1.1000,1.1000,1.100,1.1000,1.1000,1.1500,1.1500,1.1500
Interest Coverage Test - Class D,1.0100,1.050,1.0500,NaN,NaN,1.050,1.0500,NaN,1.0500,NaN,NaN,NaN,1.0500,1.1000,1.1000,1.1000
Interest Diversion Test,1.0237,1.051,NaN,NaN,NaN,1.041,NaN,1.058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Overcollateralization Ratio Test - Class A,1.2460,1.220,1.2433,1.216,1.216,1.212,1.2158,1.223,1.2233,1.2245,1.219,1.2245,NaN,NaN,NaN,NaN
Overcollateralization Ratio Test - Class B,1.1388,1.134,1.1495,1.140,1.140,1.135,1.1336,1.140,1.1495,1.1321,1.149,1.1321,NaN,NaN,NaN,NaN
Overcollateralization Ratio Test - Class A/B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2210,1.2157,1.2158,1.2513
Overcollateralization Ratio Test - Class C,1.0728,1.075,1.0796,1.076,1.076,1.086,1.0842,1.076,1.0894,1.0763,1.076,1.0764,1.1450,1.1395,1.1395,1.1750


In [107]:
#all_stats.index

all_stats_map = {'Min Floating Spread Test - no Libor Floors':'Minimum Floating Spread Test',
       'Min Floating Spread Test - With Libor Floors':'Min Floating Spread Test - With Libor Floors',
       'Minimum Weighted Average Coupon Test':'Minimum Weighted Average Coupon Test',
       'Max Moodys Rating Factor Test (NEW WARF)':'Max Moodys Rating Factor Test (NEW WARF)',
       'Max Moodys Rating Factor Test (Orig WARF)':'Maximum Moody\'s Rating Factor Test',
       'Min Moodys Recovery Rate Test':'Minimum Weighted Average Moody’s Recovery Rate Test', 
       'Min S&P Recovery Rate Class A-1a':'Minimum Weighted Average S&P Recovery Rate Test - Class A-1',
       'Moodys Diversity Test':'Moody\'s Diversity Test', 
       'Weighted Average Life Test':'Weighted Average Life',
       'Percent Caa & lower':'Moody\'s Rating <= Caa1', 
       'Percent CCC & lower':'S&P Rating <= CCC+', 
       'Percent 2nd Lien':'Percent 2nd Lien',
       'Total Portfolio Par (excl. Defaults)':'Total Portfolio Par (excl. Defaults)',
       'Class A/B Overcollateralization Ratio':'Overcollateralization Ratio Test - Class A/B',
       'Class C Overcollateralization Ratio':'Overcollateralization Ratio Test - Class C',
       'Class D Overcollateralization Ratio':'Overcollateralization Ratio Test - Class D',
       'Class E Overcollateralization Ratio':'Overcollateralization Ratio Test - Class E',
       'Reinvestment Overcollateralization Ratio':'Reinvestment Overcollateralization Test',
       'S&P Weighted Average Rating Factor (SP WARF)':'S&P Weighted Average Rating Factor (SP WARF)',
       'Default Rate Dispersion (DRD)':'Default Rate Dispersion (DRD)', 
       'Obligor Diversity Measure (ODM)':'Obligor Diversity Measure (ODM)',
       'Industry Diversity Measure (IDM)':'Industry Diversity Measure (IDM)', 
       'Regional Diversity Measure (RDM)':'Regional Diversity Measure (RDM)',
       'Weighted Average Life (WAL)':'Weighted Average Life (WAL)', 
       'Break-Even Default Rate (BDR)':'Break-Even Default Rate (BDR)',
       'Adjusted Break-Even Default Rate (Adj BDR)':'Adjusted Break-Even Default Rate (Adj BDR)',
       'Scenario Default Rate (SDR)':'Scenario Default Rate (SDR)'}

#all_stats.loc[triggers['TestType']=='Coverage Test','Title'] = \
all_stats.index = all_stats.index.map(all_stats_map)
all_stats

,CLO 4,CLO 5,CLO 6,CLO 7,CLO 8R,CLO 9,CLO 10,CLO 11,CLO 12,CLO 13,CLO 14,CLO 15,CLO 16,CLO 17,CLO 18,CLO 19,CLO 20,CLO 21
Minimum Floating Spread Test,3.43%,3.66%,3.61%,3.67%,3.62%,3.16%,3.17%,3.60%,3.36%,3.61%,3.61%,3.59%,3.61%,3.59%,3.57%,3.58%,3.66%,3.68%
Min Floating Spread Test - With Libor Floors,3.59%,3.84%,3.79%,3.85%,3.83%,3.38%,3.37%,3.78%,3.52%,3.79%,3.79%,3.78%,3.81%,3.77%,3.75%,3.75%,3.85%,3.88%
Minimum Weighted Average Coupon Test,nan%,nan%,nan%,nan%,4.20%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,4.57%,nan%,4.30%,nan%,4.31%,4.12%
Max Moodys Rating Factor Test (NEW WARF),2853,2890,2848,2877,2830,2827,2826,2802,2809,2818,2828,2791,2815,2761,2757,2741,2762,2731
Maximum Moody's Rating Factor Test,3052,3124,3083,3118,3094,3083,3045,3030,3006,3038,3047,3001,3030,2967,2968,2948,2948,2906
Minimum Weighted Average Moody’s Recovery Rate Test,49.1%,48.1%,48.1%,48.1%,47.5%,48.9%,48.9%,48.1%,48.8%,48.1%,48.2%,48.2%,48.1%,48.2%,47.9%,47.8%,47.7%,48.1%
Minimum Weighted Average S&P Recovery Rate Test - Class A-1,42.7%,39.9%,39.9%,40.1%,39.3%,41.7%,42.0%,40.2%,41.2%,39.8%,39.8%,39.9%,40.0%,40.7%,40.6%,40.1%,40.0%,40.7%
Moody's Diversity Test,74,87,89,88,78,47,53,87,74,89,89,89,86,87,81,80,81,78
Weighted Average Life,3.94,4.89,4.86,4.79,4.86,2.98,3.03,4.84,4.07,4.85,4.90,4.93,4.97,5.06,5.15,5.14,5.26,5.36
Percent Caa & lower,6.8%,7.6%,7.2%,7.2%,5.6%,6.7%,8.5%,6.8%,6.9%,6.7%,6.1%,5.0%,5.1%,4.4%,3.6%,2.8%,2.4%,1.7%


In [109]:
trigger_table = trigger_table_q.append(trigger_table_cov)


In [110]:
#df1 = pd.DataFrame({'A':['A0','A1','A2'],'B':['B0','B1','B2'],'C':['C0','C1','C2']},index=pd.date_range('2017-01-01',periods=3, freq='M'))
#df2 = pd.DataFrame({'A':['A3','A4','A5'],'B':['B3','B4','B5'],'C':['C3','C4','C5']},index=pd.date_range('2017-01-01',periods=3, freq='M'))

pd.concat([all_stats,trigger_table],axis=1,keys=['Result','Trigger']).swaplevel(0,1,axis=1).sort_index(axis=1)

CLO 10             \
                                                         Result    Trigger   
Minimum Floating Spread Test                              3.17%     0.0300   
Min Floating Spread Test - With Libor Floors              3.37%        NaN   
Minimum Weighted Average Coupon Test                       nan%     0.0650   
Max Moodys Rating Factor Test (NEW WARF)                   2826        NaN   
Maximum Moody's Rating Factor Test                         3045  3200.0000   
Minimum Weighted Average Moody’s Recovery Rate ...        48.9%     0.4300   
Minimum Weighted Average S&P Recovery Rate Test...        42.0%     0.4000   
Moody's Diversity Test                                       53    65.0000   
Weighted Average Life                                      3.03     2.3300   
Percent Caa & lower                                        8.5%        NaN   
Percent CCC & lower                                        0.4%        NaN   
Percent 2nd Lien                                           0.3%        NaN   
Total Portfolio Par (excl. Defaults)                273,658,246        NaN   
Overcollateralization Ratio Test - Class A/B           90.0192%        NaN   
Overcollateralization Ratio Test - Class C             83.4324%     1.0842   
Overcollateralization Ratio Test - Class D             77.7438%     1.0470   
Overcollateralization Ratio Test - Class E             75.1808%        NaN   
Reinvestment Overcollateralization Test                75.1808%     1.0288   
S&P Weighted Average Rating Factor (SP WARF)        2682.824241        NaN   
Default Rate Dispersion (DRD)                        923.782354        NaN   
Obligor Diversity Measure (ODM)                       84.403261        NaN   
Industry Diversity Measure (IDM)                       16.46157        NaN   
Regional Diversity Measure (RDM)                       1.296294        NaN   
Weighted Average Life (WAL)                            3.027066        NaN   
Break-Even Default Rate (BDR)                          0.679341        NaN   
Adjusted Break-Even Default Rate (Adj BDR)             0.922388        NaN   
Scenario Default Rate (SDR)                            0.539239        NaN   
Interest Coverage Test - Class A                            NaN     1.2000   
Interest Coverage Test - Class B                            NaN     1.1500   
Interest Coverage Test - Class A/B                          NaN        NaN   
Interest Coverage Test - Class C                            NaN     1.1000   
Interest Coverage Test - Class D                            NaN     1.0500   
Interest Diversion Test                                     NaN        NaN   
Overcollateralization Ratio Test - Class A                  NaN     1.2158   
Overcollateralization Ratio Test - Class B                  NaN     1.1336   
Overcollateralization Ratio Test - Class A1 [Ev...          NaN        NaN   
Overcollateralization Ratio Test - Event of Def...          NaN     1.0250   

                                                         CLO 11            \
                                                         Result   Trigger   
Minimum Floating Spread Test                              3.60%     0.035   
Min Floating Spread Test - With Libor Floors              3.78%       NaN   
Minimum Weighted Average Coupon Test                       nan%     0.065   
Max Moodys Rating Factor Test (NEW WARF)                   2802       NaN   
Maximum Moody's Rating Factor Test                         3030  3037.000   
Minimum Weighted Average Moody’s Recovery Rate ...        48.1%     0.430   
Minimum Weighted Average S&P Recovery Rate Test...        40.2%     0.380   
Moody's Diversity Test                                       87    80.000   
Weighted Average Life                                      4.84     5.130   
Percent Caa & lower                                        6.8%       NaN   
Percent CCC & lower                                        1.8%       NaN   
Percent 2nd L

In [183]:
importlib.reload(ccu)
all_df = ccu.master_test_stats(clo_df)

<module 'CLOComplianceUtils' from 'C:\\Users\\jknechtel\\Miniconda3\\RM-IT\\CLOComplianceUtils.py'>

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:464: RuntimeWarning: invalid value encountered in double_scalars
  wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols

KeyError: 'Cov Lite'

In [182]:
all_df

CLO 10             \
                                                          Result    Trigger   
Minimum Floating Spread Test                        3.170174e+00     0.0300   
Min Floating Spread Test - With Libor Floors        3.398004e+00        NaN   
Minimum Weighted Average Coupon Test                         NaN     0.0650   
Max Moodys Rating Factor Test (NEW WARF)            2.827336e+03        NaN   
Maximum Moody's Rating Factor Test                  3.051396e+03  3200.0000   
Minimum Weighted Average Moody’s Recovery Rate ...  4.891859e+01     0.4300   
Minimum Weighted Average S&P Recovery Rate Test...  4.200253e+01     0.4000   
Moody's Diversity Test                              5.229010e+01    65.0000   
Weighted Average Life                               3.022005e+00     2.3300   
Moody's Rating <= Caa1                              8.547217e+00     0.0750   
S&P Rating <= CCC+                                  4.440817e-01     0.0750   
Percent 2nd Lien                                    2.873515e-01        NaN   
Total Portfolio Par (excl. Defaults)                2.735645e+08        NaN   
Overcollateralization Ratio Test - Class A/B        8.998833e-01        NaN   
Overcollateralization Ratio Test - Class C          8.340381e-01     1.0842   
Overcollateralization Ratio Test - Class D          7.771719e-01     1.0470   
Overcollateralization Ratio Test - Class E          7.515509e-01        NaN   
Reinvestment Overcollateralization Test             7.515509e-01     1.0288   
S&P Weighted Average Rating Factor (SP WARF)        2.683198e+03        NaN   
Default Rate Dispersion (DRD)                       9.238057e+02        NaN   
Obligor Diversity Measure (ODM)                     8.441774e+01        NaN   
Industry Diversity Measure (IDM)                    1.576654e+01        NaN   
Regional Diversity Measure (RDM)                    1.295201e+00        NaN   
Weighted Average Life (WAL)                         3.022005e+00        NaN   
Break-Even Default Rate (BDR)                       6.797013e-01        NaN   
Adjusted Break-Even Default Rate (Adj BDR)          9.237591e-01        NaN   
Scenario Default Rate (SDR)                         5.394433e-01        NaN   
Interest Coverage Test - Class A                             NaN     1.2000   
Interest Coverage Test - Class B                             NaN     1.1500   
Interest Coverage Test - Class A/B                           NaN        NaN   
Interest Coverage Test - Class C                             NaN     1.1000   
Interest Coverage Test - Class D                             NaN     1.0500   
Interest Diversion Test                                      NaN        NaN   
Overcollateralization Ratio Test - Class A                   NaN     1.2158   
Overcollateralization Ratio Test - Class B                   NaN     1.1336   
Overcollateralization Ratio Test - Event of Def...           NaN     1.0250   
Cov-Lite Loans                                               NaN     0.5000   
Moody's Default Probability Rating <= Caa1 and/...           NaN        NaN   

                                                          CLO 11            \
                                                          Result   Trigger   
Minimum Floating Spread Test                        3.616592e+00     0.035   
Min Floating Spread Test - With Libor Floors        3.825925e+00       NaN   
Minimum Weighted Average Coupon Test                         NaN     0.065   
Max Moodys Rating Factor Test (NEW WARF)            2.822418e+03       NaN   
Maximum Moody's Rating Factor Test                  3.019179e+03  3037.000   
Minimum Weighted Average Moody’s Recovery Rate ...  4.808241e+01     0.430   
Minimum Weighted Average S&P Recovery Rate Test...  4.012360e+01     0.380   
Moody's Diversity Test                              8.624670e+01    80.000   
Weighted Average Life                               4.851463e+00     5.130   
Moody's Rating <= Caa1                 

In [180]:
ccu.all_stats_all_clos(clo_df, cols=ccu.clo_list, clo_dict= dict_2020_20, format_output=False)

C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:464: RuntimeWarning: invalid value encountered in double_scalars
  wa = (model_df[cols[0]]*model_df[cols[1]]).sum()/model_df[cols

OP:  600000000.0 , NP:  556545498.6028999
Pass
OP:  400000000.0 , NP:  385234450.36660004
Pass
OP:  1000000000.0 , NP:  944389395.1255
Pass
OP:  500000000.0 , NP:  493850641.283
Pass
OP:  400000000.0 , NP:  348461546.4984


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['S&P CLO Specified Assets'] = clo_df[col].values
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:504: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Absolute Deviation'] = abs(clo_df['S&P Global Ratings Factor']-SPWARF)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

Pass
OP:  300000000.0 , NP:  307753019.81210005
Pass
OP:  300000000.0 , NP:  272349659.8843
Pass
OP:  500000000.0 , NP:  496021250.339
Pass
OP:  500000000.0 , NP:  464127618.0163
Pass


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:504: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Absolute Deviation'] = abs(clo_df['S&P Global Ratings Factor']-SPWARF)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

OP:  600000000.0 , NP:  586174183.1503
Pass
OP:  600000000.0 , NP:  593988113.7992
Pass
OP:  600000000.0 , NP:  588618092.2107999
Pass
OP:  500000000.0 , NP:  492030239.3079001
Pass
OP:  500000000.0 , NP:  492841859.86980003
Pass


C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:749: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clo_df['Average Life']  = ((clo_df['Maturity Date'] - pd.Timestamp.today()).dt.days/365).round(2)
C:\Users\jknechtel\Miniconda3\RM-IT\CLOComplianceUtils.py:738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S&P CLO Specified Assets'] = df[col].values
C:\Users\jknechtel\Miniconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

OP:  400000000.0 , NP:  397363974.5871
Pass
OP:  400000000.0 , NP:  396614583.9613
Pass
OP:  400000000.0 , NP:  397586678.6159
Pass
OP:  500000000.0 , NP:  490333310.6511
Pass


,CLO 4,CLO 5,CLO 6,CLO 7,CLO 8R,CLO 9,CLO 10,CLO 11,CLO 12,CLO 13,CLO 14,CLO 15,CLO 16,CLO 17,CLO 18,CLO 19,CLO 20,CLO 21
Min Floating Spread Test - no Libor Floors,3.424932e+00,3.670058e+00,3.624329e+00,3.684084e+00,3.649989e+00,3.161210e+00,3.170174e+00,3.616592e+00,3.356764e+00,3.620464e+00,3.622993e+00,3.602163e+00,3.631762e+00,3.598789e+00,3.583201e+00,3.598426e+00,3.669090e+00,3.710512e+00
Min Floating Spread Test - With Libor Floors,3.596993e+00,3.878258e+00,3.835389e+00,3.897487e+00,3.870385e+00,3.395285e+00,3.398004e+00,3.825925e+00,3.529562e+00,3.827638e+00,3.825191e+00,3.817500e+00,3.858546e+00,3.815684e+00,3.802312e+00,3.809282e+00,3.890123e+00,3.942157e+00
Minimum Weighted Average Coupon Test,NaN,NaN,NaN,NaN,3.875000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.451250e+00,NaN,4.301471e+00,NaN,4.005435e+00,4.125000e+00
Max Moodys Rating Factor Test (NEW WARF),2.855704e+03,2.912312e+03,2.870315e+03,2.901870e+03,2.834674e+03,2.829751e+03,2.827336e+03,2.822418e+03,2.812146e+03,2.836126e+03,2.853736e+03,2.814948e+03,2.846310e+03,2.782887e+03,2.788485e+03,2.775237e+03,2.778802e+03,2.739803e+03
Max Moodys Rating Factor Test (Orig WARF),3.064512e+03,3.112386e+03,3.071279e+03,3.109193e+03,3.077384e+03,3.093251e+03,3.051396e+03,3.019179e+03,3.016756e+03,3.024764e+03,3.041018e+03,2.993116e+03,3.026629e+03,2.954232e+03,2.968009e+03,2.947880e+03,2.929145e+03,2.890153e+03
Min Moodys Recovery Rate Test,4.908747e+01,4.811542e+01,4.809147e+01,4.806078e+01,4.760343e+01,4.889290e+01,4.891859e+01,4.808241e+01,4.880915e+01,4.805513e+01,4.816594e+01,4.816080e+01,4.809943e+01,4.816087e+01,4.792550e+01,4.785368e+01,4.773777e+01,4.796853e+01
Min S&P Recovery Rate Class A-1a,4.269515e+01,3.984427e+01,3.974020e+01,4.009012e+01,3.957175e+01,4.165399e+01,4.200253e+01,4.012360e+01,4.114236e+01,3.972748e+01,3.989916e+01,3.987702e+01,3.994877e+01,4.067488e+01,4.040309e+01,3.999211e+01,3.981928e+01,4.036557e+01
Moodys Diversity Test,7.348330e+01,8.576840e+01,8.717990e+01,8.658510e+01,7.745830e+01,4.639660e+01,5.229010e+01,8.624670e+01,7.277010e+01,8.790830e+01,8.777830e+01,8.771510e+01,8.486010e+01,8.540000e+01,8.101990e+01,7.965990e+01,8.106000e+01,7.743010e+01
Weighted Average Life Test,3.933146e+00,4.881164e+00,4.853161e+00,4.772792e+00,4.889515e+00,2.977121e+00,3.022005e+00,4.851463e+00,4.061799e+00,4.856042e+00,4.901472e+00,4.935434e+00,4.975904e+00,5.056213e+00,5.164119e+00,5.143448e+00,5.253943e+00,5.389823e+00
Percent Caa & lower,6.844568e+00,7.262873e+00,6.752841e+00,6.793890e+00,5.371849e+00,6.726752e+00,8.547217e+00,6.393259e+00,6.865772e+00,6.318442e+00,5.728405e+00,4.689522e+00,4.748963e+00,4.038093e+00,3.220387e+00,2.381071e+00,2.036717e+00,1.181152e+00


# Read the Trustee files (old)

In [281]:
path = 'Z:/Shared/Risk Management and Investment Technology/CLO Optimization/'
file_prefix = 'Portfolio Assets_ Trustee Recon Detail'
suffixes = [' (CLO 2020-18)',' (CLO 2013-4)',' (CLO 2014-6)',' (CLO 2020-19)',
            ' (CLO 2020-20)',' (CLO 2014-7)',' (CLO 2015-9)',' (CLO 2020-8r)',
            ' (CLO 2016-12)',' (CLO 2017-13)',' (CLO 2017-14)',' (CLO 2018-15)',
            ' (CLO 2019-16)',' (CLO 2019-17)']

def get_trustee_recon(filepath):
    master_df = pd.read_csv(filepath,header=0)
    master_df = master_df.loc[:,~master_df.columns.str.match("Unnamed")]
    master_df.rename(columns={'LoanX Id':'LXID'},inplace=True)
    master_df.set_index('LXID', inplace=True)
    return master_df

In [3]:
# Portfolio Assets_ Trustee Recon Detail (CLO 2020-18).CSV
filepath = path+file_prefix+suffixes[13]+'.CSV'
trustee_df = get_trustee_recon(filepath)
#
#filepath = path+file_prefix+suffixes[1]+'.CSV'
#temp = get_trustee_recon(filepath)
trustee_df['Maturity Date'] = pd.to_datetime(trustee_df['Maturity Date'], infer_datetime_format=True)
trustee_df = pd.merge_asof(trustee_df.sort_values('Maturity Date'), WAL_val, left_on='Maturity Date', 
                          right_on=WAL_val.index, direction='backward', suffixes=['', '_2'])

trustee_df[['Issuer','Maturity Date','Value']]

NameError: name 'get_trustee_recon' is not defined

In [98]:
#trustee_df.merge(temp,left_on="LXID",right_on="LXID",how='left',suffixes=(suffixes[0], suffixes[1])).info()